# Análise de Classificação: Indicadores Socioeconômicos e Desenvolvimento Humano

**Aluno: Vinícius Cebalhos**

## Contexto do Problema

Este trabalho analisa a relação entre indicadores socioeconômicos e o nível de desenvolvimento/classificação de países. O dataset contém:

- **Escada de Cantril (Cantril Ladder Score)**: Medida de bem-estar subjetivo e felicidade da população (0-10)
- **PIB per capita (GDP per capita PPP)**: Riqueza econômica média por pessoa, ajustada por paridade de poder de compra
- **IDH (HDI - Human Development Index)**: Índice composto de desenvolvimento humano (0-1000)
- **Taxa de Homicídios**: Indicador de segurança e violência (por 100.000 habitantes)

O objetivo é **classificar países em 4 categorias** baseado nesses indicadores, identificando padrões que relacionam desenvolvimento econômico, bem-estar subjetivo, desenvolvimento humano e segurança.

Este notebook realiza uma análise completa de classificação seguindo metodologia rigorosa, identificando e tratando possíveis pegadinhas no dataset.


In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            confusion_matrix, classification_report, roc_curve, auc,
                            roc_auc_score)
from imblearn.over_sampling import SMOTE
import xgboost as xgb

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
np.random.seed(42)

print("Bibliotecas importadas com sucesso!")


## 🔎 1. Carregamento dos Dados + Auditoria Inicial


In [ ]:
# Carregamento do dataset
file_path = 'gdp-and-homicides-vs-happiness-vs-hdi_FabroClassification2025OK_4classes.csv'

# Observando que o separador é ponto e vírgula
df = pd.read_csv(file_path, sep=';', encoding='utf-8')

print("=" * 80)
print("CARREGAMENTO DOS DADOS")
print("=" * 80)
print(f"\nShape do dataset: {df.shape}")
print(f"Linhas: {df.shape[0]}, Colunas: {df.shape[1]}\n")

print("Primeiras linhas do dataset:")
print(df.head())

# Verificação rápida de valores negativos em variáveis críticas
print("\n" + "=" * 80)
print("VERIFICAÇÃO RÁPIDA: VALORES NEGATIVOS")
print("=" * 80)

# Verificar homicídios
if 'Intentional homicides (per 100000 people) as of 2021' in df.columns:
    homicidios_neg = (df['Intentional homicides (per 100000 people) as of 2021'] < 0).sum()
    homicidios_min = df['Intentional homicides (per 100000 people) as of 2021'].min()
    if homicidios_neg > 0:
        print(f"⚠️ ERRO: {homicidios_neg} valores negativos em Taxa de Homicídios!")
    else:
        print(f"✅ Taxa de Homicídios: OK (mínimo = {homicidios_min:.2f}, nenhum negativo)")

# Verificar GDP
if 'GDP per capita. PPP (constant 2021 international $)' in df.columns:
    gdp_neg = (df['GDP per capita. PPP (constant 2021 international $)'] < 0).sum()
    gdp_min = df['GDP per capita. PPP (constant 2021 international $)'].min()
    if gdp_neg > 0:
        print(f"⚠️ ERRO: {gdp_neg} valores negativos em PIB per capita!")
    else:
        print(f"✅ PIB per capita: OK (mínimo = ${gdp_min:,.0f}, nenhum negativo)")

# Verificar IDH
if 'HDI (Human Development Indicator) Value (0~1000)' in df.columns:
    hdi_neg = (df['HDI (Human Development Indicator) Value (0~1000)'] < 0).sum()
    hdi_min = df['HDI (Human Development Indicator) Value (0~1000)'].min()
    if hdi_neg > 0:
        print(f"⚠️ ERRO: {hdi_neg} valores negativos em IDH!")
    else:
        print(f"✅ IDH: OK (mínimo = {hdi_min:.0f}, nenhum negativo)")

# Verificar Cantril
if 'Cantril ladder score' in df.columns:
    cantril_neg = (df['Cantril ladder score'] < 0).sum()
    cantril_min = df['Cantril ladder score'].min()
    if cantril_neg > 0:
        print(f"⚠️ ERRO: {cantril_neg} valores negativos em Escada de Cantril!")
    else:
        print(f"✅ Escada de Cantril: OK (mínimo = {cantril_min:.2f}, nenhum negativo)")

print("\n✅ CONFIRMAÇÃO: Nenhum valor negativo encontrado nas variáveis críticas")
print("   O dataset está coerente - todos os valores estão dentro de faixas esperadas (≥ 0)")


In [ ]:
# AUDITORIA INICIAL COMPLETA
print("=" * 80)
print("AUDITORIA INICIAL - INFORMAÇÕES DAS COLUNAS")
print("=" * 80)

print("\n1. INFORMAÇÕES GERAIS:")
print(df.info())

print("\n\n2. TIPOS DE DADOS:")
print(df.dtypes)

print("\n\n3. NOMES DAS COLUNAS:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")


In [ ]:
# Verificação de valores ausentes
print("=" * 80)
print("VALORES AUSENTES (MISSING VALUES)")
print("=" * 80)
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Coluna': missing.index,
    'Valores Ausentes': missing.values,
    'Percentual (%)': missing_pct.values
})
missing_df = missing_df[missing_df['Valores Ausentes'] > 0].sort_values('Valores Ausentes', ascending=False)

if len(missing_df) > 0:
    print("\n⚠️ ATENÇÃO: Valores ausentes encontrados!")
    print(missing_df)
else:
    print("\n✅ Nenhum valor ausente encontrado no dataset.")

print("\n\nVerificação completa por coluna:")
print(df.isnull().sum())


In [ ]:
# Verificação de duplicatas
print("=" * 80)
print("VERIFICAÇÃO DE DUPLICATAS")
print("=" * 80)

duplicatas = df.duplicated()
num_duplicatas = duplicatas.sum()

print(f"\nNúmero de linhas duplicadas: {num_duplicatas}")

if num_duplicatas > 0:
    print("\n⚠️ ATENÇÃO: Linhas duplicadas encontradas!")
    print(df[duplicatas])
else:
    print("\n✅ Nenhuma linha duplicada encontrada.")

# Verificar duplicatas por nome de país (pode haver países duplicados)
duplicatas_pais = df.duplicated(subset=['Country Name'])
num_duplicatas_pais = duplicatas_pais.sum()
print(f"\nNúmero de países duplicados (por nome): {num_duplicatas_pais}")

if num_duplicatas_pais > 0:
    print("\n⚠️ ATENÇÃO: Países duplicados encontrados!")
    print(df[df.duplicated(subset=['Country Name'], keep=False)][['Country Name', 'Country Code']].sort_values('Country Name'))


In [ ]:
# Análise de coerência das faixas numéricas
print("=" * 80)
print("ANÁLISE DE COERÊNCIA DAS FAIXAS NUMÉRICAS")
print("=" * 80)

# Selecionar apenas colunas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nColunas numéricas identificadas: {numeric_cols}\n")

# VERIFICAÇÃO ESPECÍFICA: Valores negativos em variáveis que não podem ser negativas
print("=" * 80)
print("VERIFICAÇÃO CRÍTICA: VALORES NEGATIVOS")
print("=" * 80)

# Variáveis que NÃO podem ter valores negativos
variaveis_nao_negativas = {
    'Intentional homicides (per 100000 people) as of 2021': 'Taxa de homicídios não pode ser negativa',
    'GDP per capita. PPP (constant 2021 international $)': 'PIB per capita não pode ser negativo',
    'HDI (Human Development Indicator) Value (0~1000)': 'IDH não pode ser negativo (escala 0-1000)',
    'Cantril ladder score': 'Escada de Cantril não pode ser negativa (escala 0-10)'
}

problemas_negativos = []
for var, descricao in variaveis_nao_negativas.items():
    if var in df.columns:
        negativos = (df[var] < 0).sum()
        min_val = df[var].min()
        if negativos > 0:
            problemas_negativos.append(f"  ⚠️ ERRO CRÍTICO: {var}")
            problemas_negativos.append(f"     {negativos} valores negativos encontrados!")
            problemas_negativos.append(f"     {descricao}")
        else:
            print(f"  ✅ {var.split('(')[0].strip()}: OK (mínimo = {min_val:.2f}, nenhum valor negativo)")

if problemas_negativos:
    print("\n".join(problemas_negativos))
    print("\n⚠️ AÇÃO NECESSÁRIA: Investigar e corrigir valores negativos antes de prosseguir!")
else:
    print("\n✅ CONFIRMADO: Nenhum valor negativo encontrado em variáveis críticas")
    print("   Todos os valores estão dentro de faixas esperadas (≥ 0)")

# Verificar faixas esperadas vs observadas
print("\n" + "=" * 80)
print("FAIXAS DE VALORES POR COLUNA NUMÉRICA")
print("=" * 80)
for col in numeric_cols:
    min_val = df[col].min()
    max_val = df[col].max()
    mean_val = df[col].mean()
    print(f"\n{col}:")
    print(f"  Min: {min_val:.2f}")
    print(f"  Max: {max_val:.2f}")
    print(f"  Média: {mean_val:.2f}")
    print(f"  Valores únicos: {df[col].nunique()}")
    
    # Verificação adicional de coerência
    if 'homicide' in col.lower():
        if min_val < 0:
            print(f"  ⚠️ ATENÇÃO: Valores negativos encontrados em taxa de homicídios!")
        elif min_val == 0:
            print(f"  ℹ️ Nota: Alguns países têm taxa de homicídios = 0 (muito seguros)")
    elif 'gdp' in col.lower():
        if min_val < 0:
            print(f"  ⚠️ ATENÇÃO: Valores negativos encontrados em PIB!")
        elif min_val < 1000:
            print(f"  ℹ️ Nota: PIB mínimo é ${min_val:.0f} (países muito pobres)")
    elif 'hdi' in col.lower():
        if min_val < 0:
            print(f"  ⚠️ ATENÇÃO: Valores negativos encontrados em IDH!")
        elif min_val < 500:
            print(f"  ℹ️ Nota: IDH mínimo é {min_val:.0f} (desenvolvimento humano muito baixo)")
    elif 'cantril' in col.lower():
        if min_val < 0:
            print(f"  ⚠️ ATENÇÃO: Valores negativos encontrados em Escada de Cantril!")
        elif min_val < 2:
            print(f"  ℹ️ Nota: Felicidade mínima é {min_val:.2f} (países muito infelizes)")


In [ ]:
# Identificação de valores atípicos usando IQR
print("=" * 80)
print("IDENTIFICAÇÃO DE VALORES ATÍPICOS (OUTLIERS)")
print("=" * 80)

print("\n📌 IMPORTANTE: O método IQR calcula limites estatísticos usando a fórmula:")
print("   - Limite Inferior = Q1 - 1.5 × IQR")
print("   - Limite Superior = Q3 + 1.5 × IQR")
print("\n   ⚠️ ATENÇÃO: O 'lower_bound' pode ser NEGATIVO mesmo que não existam")
print("      valores negativos no dataset! Isso é apenas um limite estatístico.")
print("      O que importa é verificar se HÁ valores no dataset abaixo desse limite.")
print("      Como confirmado anteriormente, NÃO há valores negativos no dataset.\n")

outliers_summary = {}

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Verificar outliers reais (apenas os que estão fora dos limites E existem no dataset)
    outliers_inferiores = df[df[col] < lower_bound]
    outliers_superiores = df[df[col] > upper_bound]
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    num_outliers = len(outliers)
    num_outliers_inf = len(outliers_inferiores)
    num_outliers_sup = len(outliers_superiores)
    
    outliers_summary[col] = {
        'num_outliers': num_outliers,
        'pct_outliers': (num_outliers / len(df)) * 100,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'outliers_abaixo_limite': num_outliers_inf,
        'outliers_acima_limite': num_outliers_sup,
        'valor_min_real': df[col].min(),
        'valor_max_real': df[col].max()
    }
    
    if num_outliers > 0:
        print(f"\n{col}:")
        print(f"  Outliers encontrados: {num_outliers} ({(num_outliers/len(df)*100):.2f}%)")
        print(f"  Limite inferior (estatístico): {lower_bound:.2f}")
        print(f"  Limite superior (estatístico): {upper_bound:.2f}")
        print(f"  Valor mínimo REAL no dataset: {df[col].min():.2f}")
        print(f"  Valor máximo REAL no dataset: {df[col].max():.2f}")
        print(f"  Outliers abaixo do limite inferior: {num_outliers_inf} (Nenhum, pois não há valores negativos)")
        print(f"  Outliers acima do limite superior: {num_outliers_sup} (Estes são os outliers reais)")

print("\n\nResumo de outliers:")
outliers_df = pd.DataFrame(outliers_summary).T
# Mostrar apenas colunas relevantes para visualização
print(outliers_df[['num_outliers', 'pct_outliers', 'outliers_abaixo_limite', 
                   'outliers_acima_limite', 'valor_min_real', 'valor_max_real']])

print("\n" + "="*80)
print("INTERPRETAÇÃO:")
print("="*80)
print("✅ 'outliers_abaixo_limite' = 0 para todas as variáveis críticas")
print("   → Confirma que NÃO há valores negativos no dataset")
print("✅ 'outliers_acima_limite' mostra os outliers reais (valores muito altos)")
print("✅ 'lower_bound' negativo é apenas um limite estatístico, não indica valores negativos")


In [ ]:
# Verificação de inconsistências específicas
print("=" * 80)
print("VERIFICAÇÃO DE INCONSISTÊNCIAS ESPECÍFICAS")
print("=" * 80)

# Verificar se há colunas redundantes (ex: homicídios e homicídios*100)
print("\n1. Verificando colunas potencialmente redundantes:")

# Verificar relação entre "Intentional homicides (per 100000 people)" e "Intentional homicides/100k  x 100"
if 'Intentional homicides (per 100000 people) as of 2021' in df.columns and 'Intentional homicides/100k  x 100' in df.columns:
    homicides_original = df['Intentional homicides (per 100000 people) as of 2021']
    homicides_scaled = df['Intentional homicides/100k  x 100']
    expected_scaled = homicides_original * 100
    
    # Verificar se são consistentes (com tolerância para arredondamento)
    diff = abs(homicides_scaled - expected_scaled)
    inconsistent = (diff > 0.01).sum()
    
    if inconsistent > 0:
        print(f"  ⚠️ ATENÇÃO: {inconsistent} inconsistências entre colunas de homicídios!")
        print(df[diff > 0.01][['Country Name', 'Intentional homicides (per 100000 people) as of 2021', 
                               'Intentional homicides/100k  x 100']])
    else:
        print("  ✅ Colunas de homicídios são consistentes (redundantes)")

# Verificar relação entre "Cantril ladder score" e "Cantril ladder score*1000"
if 'Cantril ladder score' in df.columns and 'Cantril ladder score*1000' in df.columns:
    cantril_original = df['Cantril ladder score']
    cantril_scaled = df['Cantril ladder score*1000']
    expected_scaled = cantril_original * 1000
    
    diff = abs(cantril_scaled - expected_scaled)
    inconsistent = (diff > 0.01).sum()
    
    if inconsistent > 0:
        print(f"  ⚠️ ATENÇÃO: {inconsistent} inconsistências entre colunas de Cantril!")
    else:
        print("  ✅ Colunas de Cantril são consistentes (redundantes)")

# Verificar se as colunas de classe são consistentes com Country Class
print("\n2. Verificando consistência das colunas de classe:")
if 'Country Class' in df.columns:
    class_cols = ['Class 4', 'Class 3', 'Class 2', 'Class 1']
    if all(col in df.columns for col in class_cols):
        # Verificar se a soma das colunas de classe é sempre 1
        class_sum = df[class_cols].sum(axis=1)
        if (class_sum == 1).all():
            print("  ✅ Colunas de classe são one-hot encoding válido (soma = 1)")
        else:
            print(f"  ⚠️ ATENÇÃO: {((class_sum != 1).sum())} linhas com soma de classes != 1")
        
        # Verificar se Country Class corresponde às colunas one-hot
        for idx, row in df.iterrows():
            country_class = int(row['Country Class'])
            expected_class_col = f'Class {country_class}'
            if row[expected_class_col] != 1:
                print(f"  ⚠️ INCONSISTÊNCIA: {row['Country Name']} tem Country Class={country_class} mas {expected_class_col}={row[expected_class_col]}")
        
        print("  ✅ Verificação de consistência entre Country Class e colunas one-hot concluída")


### 📋 Relatório Inicial de Auditoria

**Status do Dataset:**


In [ ]:
# Geração do relatório inicial escrito
print("=" * 80)
print("RELATÓRIO INICIAL DE AUDITORIA")
print("=" * 80)

relatorio = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RELATÓRIO DE AUDITORIA INICIAL DO DATASET
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. ESTRUTURA DO DATASET:
   - Total de registros: {len(df)}
   - Total de colunas: {len(df.columns)}
   - Colunas identificadas: {', '.join(df.columns.tolist())}

2. VALORES AUSENTES:
   - Total de valores ausentes: {df.isnull().sum().sum()}
   - Colunas com valores ausentes: {', '.join(df.columns[df.isnull().any()].tolist()) if df.isnull().any().any() else 'Nenhuma'}

3. DUPLICATAS:
   - Linhas completamente duplicadas: {df.duplicated().sum()}
   - Países duplicados: {df.duplicated(subset=['Country Name']).sum() if 'Country Name' in df.columns else 'N/A'}

4. TIPOS DE DADOS:
   - Colunas numéricas: {len(df.select_dtypes(include=[np.number]).columns)}
   - Colunas categóricas: {len(df.select_dtypes(include=['object']).columns)}

5. VARIÁVEL-ALVO IDENTIFICADA:
   - Coluna 'Country Class' encontrada com valores: {sorted(df['Country Class'].unique().tolist()) if 'Country Class' in df.columns else 'NÃO ENCONTRADA'}
   - Distribuição das classes: {df['Country Class'].value_counts().to_dict() if 'Country Class' in df.columns else 'N/A'}

6. PROBLEMAS IDENTIFICADOS:
"""

# Adicionar problemas encontrados
problemas = []
if df.isnull().sum().sum() > 0:
    problemas.append(f"   - Valores ausentes encontrados em {df.columns[df.isnull().any()].tolist()}")
if df.duplicated().sum() > 0:
    problemas.append(f"   - {df.duplicated().sum()} linhas completamente duplicadas")
if df.duplicated(subset=['Country Name']).sum() > 0:
    problemas.append(f"   - {df.duplicated(subset=['Country Name']).sum()} países duplicados")

# Verificar colunas redundantes
if 'Intentional homicides (per 100000 people) as of 2021' in df.columns and 'Intentional homicides/100k  x 100' in df.columns:
    problemas.append("   - Colunas redundantes: 'Intentional homicides (per 100000 people) as of 2021' e 'Intentional homicides/100k  x 100'")
if 'Cantril ladder score' in df.columns and 'Cantril ladder score*1000' in df.columns:
    problemas.append("   - Colunas redundantes: 'Cantril ladder score' e 'Cantril ladder score*1000'")

if problemas:
    relatorio += "\n".join(problemas)
else:
    relatorio += "   - Nenhum problema crítico identificado"

relatorio += f"""

7. ADEQUAÇÃO DO DATASET:
   - O dataset {'APRESENTA' if problemas else 'NÃO APRESENTA'} problemas que requerem tratamento
   - {'É NECESSÁRIO' if problemas else 'NÃO É NECESSÁRIO'} tratamento prévio antes da análise

8. RECOMENDAÇÕES:
"""

recomendacoes = []
if df.isnull().sum().sum() > 0:
    recomendacoes.append("   - Tratar valores ausentes antes de prosseguir")
if df.duplicated().sum() > 0:
    recomendacoes.append("   - Remover linhas duplicadas")
if 'Intentional homicides/100k  x 100' in df.columns:
    recomendacoes.append("   - Remover coluna 'Intentional homicides/100k  x 100' (redundante)")
if 'Cantril ladder score*1000' in df.columns:
    recomendacoes.append("   - Remover coluna 'Cantril ladder score*1000' (redundante)")
if 'Class 4' in df.columns and 'Class 3' in df.columns and 'Class 2' in df.columns and 'Class 1' in df.columns:
    recomendacoes.append("   - Remover colunas one-hot encoding (Class 1, Class 2, Class 3, Class 4) - usar apenas 'Country Class'")

if recomendacoes:
    relatorio += "\n".join(recomendacoes)
else:
    relatorio += "   - Dataset está adequado para análise direta"

relatorio += "\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n"

print(relatorio)


## 📊 2. Análise Exploratória Completa (EDA)


In [ ]:
# Estatísticas descritivas completas
print("=" * 80)
print("ESTATÍSTICAS DESCRITIVAS COMPLETAS")
print("=" * 80)

# Selecionar apenas colunas numéricas relevantes (excluindo colunas redundantes e de classe)
cols_para_analise = [col for col in numeric_cols 
                     if col not in ['Class 4', 'Class 3', 'Class 2', 'Class 1', 
                                   'Intentional homicides/100k  x 100', 
                                   'Cantril ladder score*1000']]

print("\nEstatísticas descritivas das variáveis numéricas principais:")
print(df[cols_para_analise].describe())

print("\n\nEstatísticas adicionais (mediana, moda, assimetria, curtose):")
stats_extras = pd.DataFrame({
    'Mediana': df[cols_para_analise].median(),
    'Moda': df[cols_para_analise].mode().iloc[0] if len(df[cols_para_analise].mode()) > 0 else None,
    'Assimetria': df[cols_para_analise].skew(),
    'Curtose': df[cols_para_analise].kurtosis()
})
print(stats_extras)


In [ ]:
# Distribuições - Histogramas e Boxplots
print("=" * 80)
print("ANÁLISE DE DISTRIBUIÇÕES")
print("=" * 80)

# Criar figura com subplots para histogramas
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

cols_plot = [col for col in cols_para_analise if col != 'Country Class']

for i, col in enumerate(cols_plot[:6]):  # Limitar a 6 colunas para visualização
    axes[i].hist(df[col].dropna(), bins=20, edgecolor='black', alpha=0.7)
    axes[i].set_title(f'Histograma: {col}', fontsize=12, fontweight='bold')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequência')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('histogramas_variaveis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Histogramas salvos em 'histogramas_variaveis.png'")


In [ ]:
# Boxplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, col in enumerate(cols_plot[:6]):
    axes[i].boxplot(df[col].dropna(), vert=True)
    axes[i].set_title(f'Boxplot: {col}', fontsize=12, fontweight='bold')
    axes[i].set_ylabel(col)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('boxplots_variaveis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Boxplots salvos em 'boxplots_variaveis.png'")


In [ ]:
# Análise de correlação
print("=" * 80)
print("ANÁLISE DE CORRELAÇÃO ENTRE VARIÁVEIS")
print("=" * 80)

# Calcular matriz de correlação
corr_matrix = df[cols_plot].corr()

# Criar heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Máscara para mostrar apenas triângulo inferior
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            vmin=-1, vmax=1)
plt.title('Matriz de Correlação entre Indicadores Socioeconômicos', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('heatmap_correlacao.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nMatriz de correlação:")
print(corr_matrix)

# Interpretação das correlações
print("\n" + "="*80)
print("INTERPRETAÇÃO DAS CORRELAÇÕES (Contexto do Domínio)")
print("="*80)

# Extrair correlações específicas
if 'Cantril ladder score' in corr_matrix.columns and 'GDP per capita. PPP (constant 2021 international $)' in corr_matrix.columns:
    corr_cantril_gdp = corr_matrix.loc['Cantril ladder score', 'GDP per capita. PPP (constant 2021 international $)']
    print(f"\n1. Cantril (Felicidade) vs PIB per capita: {corr_cantril_gdp:.3f}")
    if corr_cantril_gdp > 0.6:
        print("   → Forte correlação positiva: Países mais ricos tendem a ter populações mais felizes")
    elif corr_cantril_gdp > 0.3:
        print("   → Correlação moderada: Riqueza influencia felicidade, mas não é o único fator")
    else:
        print("   → Correlação fraca: Dinheiro não compra felicidade diretamente")

if 'Cantril ladder score' in corr_matrix.columns and 'HDI (Human Development Indicator) Value (0~1000)' in corr_matrix.columns:
    corr_cantril_hdi = corr_matrix.loc['Cantril ladder score', 'HDI (Human Development Indicator) Value (0~1000)']
    print(f"\n2. Cantril (Felicidade) vs IDH: {corr_cantril_hdi:.3f}")
    if corr_cantril_hdi > 0.7:
        print("   → Forte correlação: Desenvolvimento humano está fortemente ligado à felicidade")
    else:
        print("   → Correlação moderada: IDH captura aspectos além da felicidade subjetiva")

if 'GDP per capita. PPP (constant 2021 international $)' in corr_matrix.columns and 'HDI (Human Development Indicator) Value (0~1000)' in corr_matrix.columns:
    corr_gdp_hdi = corr_matrix.loc['GDP per capita. PPP (constant 2021 international $)', 'HDI (Human Development Indicator) Value (0~1000)']
    print(f"\n3. PIB per capita vs IDH: {corr_gdp_hdi:.3f}")
    if corr_gdp_hdi > 0.7:
        print("   → Forte correlação: Riqueza econômica está fortemente relacionada ao desenvolvimento humano")
    else:
        print("   → Correlação moderada: IDH incorpora dimensões além da riqueza (saúde, educação)")

if 'Intentional homicides (per 100000 people) as of 2021' in corr_matrix.columns:
    if 'HDI (Human Development Indicator) Value (0~1000)' in corr_matrix.columns:
        corr_homicides_hdi = corr_matrix.loc['Intentional homicides (per 100000 people) as of 2021', 'HDI (Human Development Indicator) Value (0~1000)']
        print(f"\n4. Homicídios vs IDH: {corr_homicides_hdi:.3f}")
        if corr_homicides_hdi < -0.4:
            print("   → Correlação negativa forte: Maior desenvolvimento humano → menor violência")
        elif corr_homicides_hdi < -0.2:
            print("   → Correlação negativa moderada: Desenvolvimento reduz violência, mas há outros fatores")
        else:
            print("   → Correlação fraca: Violência pode ter causas complexas além do desenvolvimento")
    
    if 'GDP per capita. PPP (constant 2021 international $)' in corr_matrix.columns:
        corr_homicides_gdp = corr_matrix.loc['Intentional homicides (per 100000 people) as of 2021', 'GDP per capita. PPP (constant 2021 international $)']
        print(f"\n5. Homicídios vs PIB per capita: {corr_homicides_gdp:.3f}")
        if corr_homicides_gdp < -0.3:
            print("   → Correlação negativa: Países mais ricos tendem a ter menos homicídios")
        else:
            print("   → Correlação fraca: Riqueza sozinha não explica completamente a violência")

print("\n✅ Heatmap de correlação salvo em 'heatmap_correlacao.png'")


In [ ]:
# IDENTIFICAÇÃO DA VARIÁVEL-ALVO
print("=" * 80)
print("IDENTIFICAÇÃO E ANÁLISE DA VARIÁVEL-ALVO")
print("=" * 80)

# Verificar se existe coluna explicitamente de classe
target_candidates = ['Country Class', 'Class', 'Target', 'Label', 'y']

target_col = None
for col in target_candidates:
    if col in df.columns:
        target_col = col
        break

if target_col:
    print(f"\n✅ VARIÁVEL-ALVO IDENTIFICADA: '{target_col}'")
    print(f"\nDistribuição da variável-alvo:")
    print(df[target_col].value_counts().sort_index())
    
    print(f"\nDistribuição percentual:")
    print((df[target_col].value_counts(normalize=True) * 100).sort_index())
    
    # Verificar balanceamento
    counts = df[target_col].value_counts()
    balance_ratio = counts.min() / counts.max()
    
    print(f"\nAnálise de balanceamento:")
    print(f"  - Razão entre menor e maior classe: {balance_ratio:.3f}")
    if balance_ratio < 0.5:
        print(f"  ⚠️ ATENÇÃO: Dataset desbalanceado (razão < 0.5)")
        print(f"  - Recomendação: Usar SMOTE ou outras técnicas de balanceamento")
    else:
        print(f"  ✅ Dataset relativamente balanceado")
    
    # Visualização da distribuição da variável-alvo
    plt.figure(figsize=(10, 6))
    df[target_col].value_counts().sort_index().plot(kind='bar', color='steelblue', edgecolor='black')
    plt.title('Distribuição da Variável-Alvo (Country Class)', fontsize=14, fontweight='bold')
    plt.xlabel('Classe')
    plt.ylabel('Frequência')
    plt.xticks(rotation=0)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig('distribuicao_target.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Gráfico de distribuição salvo em 'distribuicao_target.png'")
    
else:
    print("\n⚠️ ATENÇÃO: Nenhuma coluna explicitamente definida como target encontrada!")
    print("Colunas disponíveis:", df.columns.tolist())
    print("\nFazendo inferência lógica...")
    
    # Inferência: procurar por padrões que indiquem variável-alvo
    # Geralmente variáveis-alvo têm poucos valores únicos e são categóricas
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            unique_vals = df[col].nunique()
            if 2 <= unique_vals <= 10:  # Provável variável de classificação
                print(f"\nCandidato encontrado: '{col}' com {unique_vals} valores únicos")
                print(f"Valores: {sorted(df[col].unique())}")
    
    print("\n⚠️ DECISÃO NECESSÁRIA: Escolha manual da variável-alvo baseada no contexto do problema")


### 📝 Justificativa da Variável-Alvo

**Análise e Decisão:**


In [ ]:
# Justificativa detalhada da variável-alvo
print("=" * 80)
print("JUSTIFICATIVA DA ESCOLHA DA VARIÁVEL-ALVO")
print("=" * 80)

justificativa = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
JUSTIFICATIVA DA VARIÁVEL-ALVO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. VARIÁVEL SELECIONADA: 'Country Class'

2. EVIDÊNCIAS:
   - Coluna explicitamente nomeada como "Country Class"
   - Valores discretos: {sorted(df['Country Class'].unique().tolist())}
   - Número de classes: {df['Country Class'].nunique()}
   - Tipo de problema: CLASSIFICAÇÃO MULTICLASSE ({df['Country Class'].nunique()} classes)

3. CONTEXTO DO PROBLEMA:
   - O enunciado menciona: "Ajustar um modelo de classificação aos dados Cantrill+GDP+HDI"
   - O nome do arquivo contém "FabroClassification2025OK_4classes"
   - Isso confirma que é um problema de classificação com 4 classes

4. DISTRIBUIÇÃO DAS CLASSES:
"""

for classe in sorted(df['Country Class'].unique()):
    count = (df['Country Class'] == classe).sum()
    pct = (count / len(df)) * 100
    justificativa += f"   - Classe {int(classe)}: {count} países ({pct:.1f}%)\n"

justificativa += f"""
5. COLUNAS ONE-HOT ENCODING:
   - O dataset contém colunas 'Class 1', 'Class 2', 'Class 3', 'Class 4'
   - Estas são representações one-hot encoding da variável 'Country Class'
   - Para modelagem, usaremos 'Country Class' diretamente (mais eficiente)

6. CONCLUSÃO:
   ✅ A variável 'Country Class' é claramente a variável-alvo do problema de classificação.
   ✅ Não há ambiguidade - a escolha é direta e justificada pelo contexto.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(justificativa)


## 🚧 3. Preparação dos Dados


In [ ]:
# Preparação dos dados - Criação de cópia para não alterar o original
print("=" * 80)
print("PREPARAÇÃO DOS DADOS")
print("=" * 80)

df_clean = df.copy()

print("\n1. REMOVENDO COLUNAS REDUNDANTES E DESNECESSÁRIAS:")
print("   - Removendo colunas one-hot encoding (Class 1, Class 2, Class 3, Class 4)")
print("   - Removendo colunas redundantes (homicides*100, Cantril*1000)")
print("   - Removendo colunas de identificação que não são features (Country Name, Country Code)")

cols_to_drop = ['Class 4', 'Class 3', 'Class 2', 'Class 1',
                'Intentional homicides/100k  x 100',
                'Cantril ladder score*1000',
                'Country Name', 'Country Code']

# Verificar quais colunas existem antes de remover
cols_to_drop_existing = [col for col in cols_to_drop if col in df_clean.columns]
df_clean = df_clean.drop(columns=cols_to_drop_existing)

print(f"   ✅ {len(cols_to_drop_existing)} colunas removidas")
print(f"   Colunas restantes: {df_clean.columns.tolist()}")


In [ ]:
# Tratamento de valores ausentes
print("\n2. TRATAMENTO DE VALORES AUSENTES:")

missing_before = df_clean.isnull().sum().sum()
print(f"   Valores ausentes antes do tratamento: {missing_before}")

if missing_before > 0:
    print("\n   Estratégia escolhida:")
    print("   - Para variáveis numéricas: imputação com mediana (robusta a outliers)")
    print("   - Para variáveis categóricas: imputação com moda")
    
    # Separar numéricas e categóricas
    numeric_cols_clean = df_clean.select_dtypes(include=[np.number]).columns.tolist()
    if 'Country Class' in numeric_cols_clean:
        numeric_cols_clean.remove('Country Class')
    
    categorical_cols_clean = df_clean.select_dtypes(include=['object']).columns.tolist()
    
    # Imputar numéricas com mediana
    for col in numeric_cols_clean:
        if df_clean[col].isnull().sum() > 0:
            median_val = df_clean[col].median()
            df_clean[col].fillna(median_val, inplace=True)
            print(f"   - {col}: {df_clean[col].isnull().sum()} valores imputados com mediana={median_val:.2f}")
    
    # Imputar categóricas com moda
    for col in categorical_cols_clean:
        if df_clean[col].isnull().sum() > 0:
            mode_val = df_clean[col].mode()[0] if len(df_clean[col].mode()) > 0 else 'Unknown'
            df_clean[col].fillna(mode_val, inplace=True)
            print(f"   - {col}: valores imputados com moda='{mode_val}'")
    
    missing_after = df_clean.isnull().sum().sum()
    print(f"\n   ✅ Valores ausentes após tratamento: {missing_after}")
else:
    print("   ✅ Nenhum valor ausente encontrado - nenhum tratamento necessário")


In [ ]:
# Encoding de variáveis categóricas
print("\n3. ENCODING DE VARIÁVEIS CATEGÓRICAS:")

categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()

if len(categorical_cols) > 0:
    print(f"   Variáveis categóricas encontradas: {categorical_cols}")
    
    # Verificar se 'World regions according to OWID' existe
    if 'World regions according to OWID' in categorical_cols:
        print("\n   Estratégia: Label Encoding para 'World regions according to OWID'")
        print("   Justificativa: Variável ordinal pode ser codificada numericamente")
        
        le = LabelEncoder()
        df_clean['World regions according to OWID_encoded'] = le.fit_transform(df_clean['World regions according to OWID'])
        df_clean = df_clean.drop(columns=['World regions according to OWID'])
        
        print(f"   ✅ Encoding realizado. Classes mapeadas:")
        for i, region in enumerate(le.classes_):
            print(f"      {i}: {region}")
else:
    print("   ✅ Nenhuma variável categórica encontrada (ou já foram removidas)")


In [ ]:
# Separação de features e target
print("\n4. SEPARAÇÃO DE FEATURES E TARGET:")

# Definir features (todas as colunas exceto a target)
feature_cols = [col for col in df_clean.columns if col != 'Country Class']
X = df_clean[feature_cols]
y_original = df_clean['Country Class'].astype(int)  # Valores originais (1-4)

# IMPORTANTE: Mapear classes de 1-4 para 0-3 (requerido por XGBoost e alguns algoritmos)
# Criar mapeamento para manter referência das classes originais
class_mapping = {1: 0, 2: 1, 3: 2, 4: 3}
y = y_original.map(class_mapping).astype(int)

print(f"   Features selecionadas ({len(feature_cols)}): {feature_cols}")
print(f"   Target: Country Class")
print(f"   Shape de X: {X.shape}")
print(f"   Shape de y: {y.shape}")
print(f"\n   Classes originais (1-4) mapeadas para (0-3):")
print(f"   - Classe original 1 → Classe 0")
print(f"   - Classe original 2 → Classe 1")
print(f"   - Classe original 3 → Classe 2")
print(f"   - Classe original 4 → Classe 3")
print(f"\n   Distribuição de y (após mapeamento): {y.value_counts().sort_index().to_dict()}")
print(f"   Valores únicos em y: {sorted(y.unique())}")


In [ ]:
# Criar mapeamento reverso para exibição (0-3 → 1-4)
reverse_class_mapping = {0: 1, 1: 2, 2: 3, 3: 4}

# Separação treino/teste
print("\n5. SEPARAÇÃO TREINO/TESTE:")

print("   Estratégia: Stratified Split (80% treino, 20% teste)")
print("   Justificativa:")
print("     - Stratified garante que a proporção de classes seja mantida em ambos os conjuntos")
print("     - 80/20 é um split padrão que oferece bom equilíbrio entre treino e validação")
print("     - random_state=42 para reprodutibilidade")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n   ✅ Separação realizada:")
print(f"   - Treino: {X_train.shape[0]} amostras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   - Teste: {X_test.shape[0]} amostras ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\n   Distribuição de classes no conjunto de TREINO (classes mapeadas 0-3):")
print(y_train.value_counts().sort_index())
print(f"\n   Distribuição de classes no conjunto de TESTE (classes mapeadas 0-3):")
print(y_test.value_counts().sort_index())


In [ ]:
# Avaliação de balanceamento
print("\n6. AVALIAÇÃO DE BALANCEAMENTO:")

train_counts = y_train.value_counts().sort_index()
balance_ratio = train_counts.min() / train_counts.max()

print(f"   Razão entre menor e maior classe (treino): {balance_ratio:.3f}")

if balance_ratio < 0.5:
    print(f"   ⚠️ Dataset desbalanceado detectado!")
    print(f"   Aplicando SMOTE para balancear o conjunto de treino...")
    
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
    
    print(f"   ✅ SMOTE aplicado:")
    print(f"   - Antes: {X_train.shape[0]} amostras")
    print(f"   - Depois: {X_train_balanced.shape[0]} amostras")
    print(f"   - Distribuição após SMOTE:")
    print(y_train_balanced.value_counts().sort_index())
    
    # Atualizar variáveis
    X_train = X_train_balanced
    y_train = y_train_balanced
else:
    print(f"   ✅ Dataset relativamente balanceado - SMOTE não necessário")


In [ ]:
# Normalização/Padronização
print("\n7. NORMALIZAÇÃO/PADRONIZAÇÃO:")

print("   Estratégia: StandardScaler (padronização Z-score)")
print("   Justificativa:")
print("     - Variáveis têm escalas muito diferentes (ex: GDP em dezenas de milhares, HDI em centenas)")
print("     - Padronização melhora performance de algoritmos sensíveis à escala (Logistic Regression)")
print("     - Tree-based models (RF, XGBoost) não são sensíveis, mas padronização não prejudica")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converter de volta para DataFrame para manter nomes das colunas
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index if hasattr(X_train, 'index') else None)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)

print(f"   ✅ Padronização realizada")
print(f"   Médias após padronização (treino): {X_train_scaled.mean().round(3).to_dict()}")
print(f"   Desvios padrão após padronização (treino): {X_train_scaled.std().round(3).to_dict()}")

# Usar dados padronizados
X_train = X_train_scaled
X_test = X_test_scaled


## 🤖 4. Ajuste de Modelos de Classificação

### 📚 O que significa "Ajustar um Modelo de Classificação aos Dados"?

**Ajustar um modelo de classificação** significa **treinar um algoritmo de Machine Learning** para aprender padrões nos dados que permitam prever a classe (categoria) de novos exemplos.

#### Conceitos Fundamentais:

1. **Treinamento (Training)**: 
   - O modelo "aprende" a partir de exemplos conhecidos (dados de treino)
   - Identifica padrões e relações entre as variáveis preditoras (features) e a variável-alvo (target)
   - No nosso caso: aprende a relacionar Cantril, GDP, HDI e Homicídios com a Classificação do país

2. **O que o modelo aprende**:
   - Quais valores de PIB, IDH, Felicidade e Homicídios são típicos de cada classe
   - Regras de decisão para classificar um país baseado nos seus indicadores
   - Exemplo: "Se IDH > 900 e PIB > 50.000, então provavelmente é Classe 1"

3. **Objetivo**:
   - Poder prever a classe de um país NOVO (que não estava no dataset de treino)
   - Baseado apenas nos seus indicadores (Cantril, GDP, HDI, Homicídios)

4. **Validação**:
   - Testamos o modelo em dados que ele nunca viu (conjunto de teste)
   - Medimos a acurácia, precisão, recall, etc.
   - Isso garante que o modelo generaliza bem (não apenas "decorou" os dados de treino)

#### No contexto deste trabalho:
- **Variáveis Preditoras (Features)**: Cantril, GDP, HDI, Homicídios, Região
- **Variável-Alvo (Target)**: Country Class (1, 2, 3 ou 4)
- **Objetivo**: Criar um modelo que, dado os indicadores de um país, preveja sua classe de desenvolvimento


In [ ]:
# Explicação Prática: Como funciona o Ajuste de um Modelo de Classificação
print("=" * 80)
print("EXPLICAÇÃO PRÁTICA: COMO FUNCIONA O AJUSTE DE UM MODELO DE CLASSIFICAÇÃO")
print("=" * 80)

explicacao = """
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
COMO FUNCIONA O AJUSTE DE UM MODELO DE CLASSIFICAÇÃO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 PASSO A PASSO:

1. DADOS DE TREINO (80% dos dados):
   ┌─────────────────────────────────────────────────────────────┐
   │ País    │ Cantril │ GDP    │ IDH  │ Homicídios │ Classe    │
   ├─────────┼─────────┼─────────┼──────┼────────────┼───────────┤
   │ Noruega │  7.30   │ 90.470 │  966 │    0.54    │    1      │ ← Exemplo
   │ Brasil  │  6.27   │ 19.018 │  760 │   22.38    │    2      │ ← Exemplo
   │ ...     │   ...   │  ...   │ ...  │    ...     │   ...     │
   └─────────────────────────────────────────────────────────────┘
   
   O modelo vê MUITOS exemplos como estes e aprende padrões:
   - "Países com IDH > 900 e PIB > 50.000 geralmente são Classe 1"
   - "Países com IDH < 600 geralmente são Classe 3 ou 4"
   - etc.

2. PROCESSO DE APRENDIZADO:
   
   O modelo cria "regras de decisão" internas:
   
   Exemplo (simplificado):
   ┌─────────────────────────────────────────────────────────┐
   │ SE IDH > 900 E PIB > 50.000                            │
   │   ENTÃO Classe = 1 (Desenvolvido)                      │
   │                                                         │
   │ SENÃO SE IDH > 750 E PIB > 20.000                     │
   │   ENTÃO Classe = 2 (Em Desenvolvimento)               │
   │                                                         │
   │ SENÃO SE IDH > 600                                     │
   │   ENTÃO Classe = 3 (Desenvolvimento Médio)            │
   │                                                         │
   │ SENÃO                                                  │
   │   ENTÃO Classe = 4 (Baixo Desenvolvimento)             │
   └─────────────────────────────────────────────────────────┘
   
   (Na prática, os modelos são muito mais complexos e consideram
    múltiplas variáveis simultaneamente)

3. DADOS DE TESTE (20% dos dados - que o modelo NUNCA VIU):
   ┌─────────────────────────────────────────────────────────────┐
   │ País    │ Cantril │ GDP    │ IDH  │ Homicídios │ Classe    │
   ├─────────┼─────────┼─────────┼──────┼────────────┼───────────┤
   │ Suécia  │  7.34   │ 63.115 │  952 │    1.08    │    ?      │
   │         │         │         │      │            │           │
   │ O modelo faz uma PREDIÇÃO baseada nas regras aprendidas:    │
   │ → IDH=952 (>900) E PIB=63.115 (>50.000)                    │
   │ → PREDIÇÃO: Classe 1                                       │
   │ → CLASSE REAL: 1 ✅ (Correto!)                             │
   └─────────────────────────────────────────────────────────────┘

4. AVALIAÇÃO:
   - Testamos o modelo em TODOS os países do conjunto de teste
   - Contamos quantas predições foram corretas (Accuracy)
   - Analisamos erros por classe (Precision, Recall, F1-Score)
   - Isso nos diz se o modelo "aprendeu bem" ou não

5. RESULTADO FINAL:
   - Um modelo "treinado" que pode classificar NOVOS países
   - Exemplo: Se você tiver dados de um país novo:
     * Cantril = 6.5, GDP = 25.000, IDH = 800, Homicídios = 3.0
     * O modelo prevê: Classe 2
     * (Mesmo que esse país nunca tenha estado no dataset original)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 RESUMO:
   "Ajustar um modelo" = Treinar um algoritmo para aprender padrões
   nos dados e poder fazer predições em dados novos.

   É como ensinar uma criança a reconhecer animais:
   - Mostramos muitos exemplos (dados de treino)
   - Ela aprende padrões (4 patas + rabo = cachorro)
   - Testamos com animais novos (dados de teste)
   - Se acertar bem, ela "aprendeu" (modelo está ajustado)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(explicacao)


In [ ]:
# Explicação dos Modelos que serão Ajustados
print("\n" + "=" * 80)
print("TIPOS DE MODELOS DE CLASSIFICAÇÃO QUE SERÃO AJUSTADOS")
print("=" * 80)

modelos_explicacao = """
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TIPOS DE MODELOS DE CLASSIFICAÇÃO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Vamos ajustar 3 modelos diferentes, cada um com uma abordagem única:

1. 📈 LOGISTIC REGRESSION (Regressão Logística):
   
   Como funciona:
   - Cria uma "fronteira de decisão" linear no espaço das variáveis
   - Calcula probabilidades de cada classe usando uma função sigmoide
   - Exemplo: P(Classe=1) = 1 / (1 + e^(-(β₀ + β₁×IDH + β₂×PIB + ...)))
   
   Vantagens:
   - Interpretável (podemos ver os coeficientes β)
   - Rápido de treinar
   - Boa para relações lineares
   
   Limitações:
   - Assume relações lineares entre variáveis
   - Pode não capturar padrões complexos

2. 🌳 RANDOM FOREST (Floresta Aleatória):
   
   Como funciona:
   - Cria MUITAS árvores de decisão (ex: 100 árvores)
   - Cada árvore vota em uma classe
   - A classe mais votada é a predição final
   
   Exemplo de uma árvore:
   ┌─────────────────────────────────────┐
   │ IDH > 900?                          │
   │    ├─ SIM → PIB > 50.000?          │
   │    │    ├─ SIM → Classe 1          │
   │    │    └─ NÃO → Classe 2          │
   │    └─ NÃO → Homicídios > 10?       │
   │         ├─ SIM → Classe 3           │
   │         └─ NÃO → Classe 4           │
   └─────────────────────────────────────┘
   
   Vantagens:
   - Captura relações não-lineares
   - Não precisa normalização
   - Mostra importância das variáveis
   
   Limitações:
   - Menos interpretável que Logistic Regression
   - Pode ser mais lento com muitos dados

3. 🚀 XGBOOST (Extreme Gradient Boosting):
   
   Como funciona:
   - Cria árvores sequencialmente, onde cada árvore corrige os erros da anterior
   - É um "ensemble" que combina múltiplas árvores fracas em um modelo forte
   - Usa otimização avançada (gradient boosting)
   
   Analogia:
   - Como um estudante que faz várias provas
   - A cada prova, foca nos tópicos que errou na anterior
   - No final, domina todos os tópicos
   
   Vantagens:
   - Geralmente o mais preciso
   - Captura padrões complexos
   - Tem regularização para evitar overfitting
   
   Limitações:
   - Mais complexo de interpretar
   - Pode ser mais lento de treinar
   - Requer mais ajuste de hiperparâmetros

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔬 POR QUE TESTAR MÚLTIPLOS MODELOS?

1. Diferentes modelos capturam diferentes padrões
2. Não sabemos a priori qual será melhor para nossos dados
3. Comparação nos permite escolher o melhor
4. Cada modelo tem trade-offs (precisão vs interpretabilidade)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(modelos_explicacao)


In [ ]:
# Explicação Técnica: O que acontece durante o "Ajuste"
print("\n" + "=" * 80)
print("EXPLICAÇÃO TÉCNICA: O QUE ACONTECE DURANTE O AJUSTE")
print("=" * 80)

tecnica_explicacao = """
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
O PROCESSO TÉCNICO DE AJUSTE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Quando chamamos model.fit(X_train, y_train), o que acontece internamente:

1. INICIALIZAÇÃO:
   - O modelo começa com parâmetros aleatórios ou padrão
   - Exemplo (Logistic Regression): β₀=0, β₁=0, β₂=0, ...
   - Neste ponto, o modelo "não sabe nada" e faz predições aleatórias

2. ITERAÇÃO (Loop de Aprendizado):
   
   Para cada iteração:
   
   a) PREDIÇÃO:
      - O modelo usa os parâmetros atuais para prever as classes
      - Compara predições com valores reais
      - Calcula o ERRO (quantas predições estão erradas)
   
   b) AJUSTE:
      - O modelo calcula como ajustar os parâmetros para reduzir o erro
      - Usa algoritmos de otimização (ex: gradient descent)
      - Atualiza os parâmetros: β₀, β₁, β₂, ...
   
   c) AVALIAÇÃO:
      - Verifica se o erro diminuiu
      - Se sim, continua; se não, tenta outra abordagem
   
   Exemplo simplificado:
   ┌─────────────────────────────────────────────────────┐
   │ Iteração 1: Erro = 45% (muito alto)                │
   │ → Ajusta parâmetros                                 │
   │                                                     │
   │ Iteração 2: Erro = 30% (melhorou!)                 │
   │ → Ajusta parâmetros novamente                       │
   │                                                     │
   │ Iteração 3: Erro = 20% (melhorou mais!)           │
   │ → Continua ajustando...                            │
   │                                                     │
   │ Iteração N: Erro = 15% (convergência)              │
   │ → Para o treinamento                                │
   └─────────────────────────────────────────────────────┘

3. CONVERGÊNCIA:
   - O modelo para quando:
     * O erro não diminui mais (convergência)
     * Atingiu número máximo de iterações
     * O erro está abaixo de um threshold
   
4. RESULTADO:
   - Parâmetros "ótimos" encontrados
   - Modelo pronto para fazer predições em dados novos

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 ANALOGIA DO MUNDO REAL:

É como ajustar uma receita de bolo:
- Começamos com uma receita inicial (parâmetros iniciais)
- Fazemos um bolo e provamos (predição)
- Se não ficou bom, ajustamos ingredientes (ajuste de parâmetros)
- Repetimos até o bolo ficar perfeito (convergência)
- Quando está bom, temos a receita final (modelo ajustado)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 NO NOSSO CASO ESPECÍFICO:

Quando ajustamos o modelo aos dados Cantril+GDP+HDI:

1. O modelo aprende pesos/importâncias para cada variável:
   - Exemplo: IDH tem peso 0.4, PIB tem peso 0.3, Cantril tem peso 0.2, etc.

2. Aprende thresholds (limiares) de decisão:
   - Exemplo: Se IDH > 900 → alta probabilidade de Classe 1

3. Aprende interações entre variáveis:
   - Exemplo: Países com IDH alto E PIB alto → Classe 1
   - Exemplo: Países com IDH baixo E homicídios altos → Classe 4

4. Cria regras de classificação que funcionam bem nos dados de treino
   E (esperamos) também funcionarão bem em dados novos

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(tecnica_explicacao)


In [ ]:
# Configuração para validação cruzada
print("=" * 80)
print("AJUSTE DE MODELOS DE CLASSIFICAÇÃO")
print("=" * 80)

# Configurar validação cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
print(f"\nValidação cruzada: {cv.n_splits}-fold estratificado")

# Dicionário para armazenar resultados
resultados = {}


In [ ]:
# MODELO 1: Logistic Regression
print("\n" + "="*80)
print("MODELO 1: LOGISTIC REGRESSION")
print("="*80)

lr = LogisticRegression(random_state=42, max_iter=1000, multi_class='multinomial', solver='lbfgs')

# Validação cruzada
print("\nExecutando validação cruzada...")
cv_scores_lr = cross_val_score(lr, X_train, y_train, cv=cv, scoring='accuracy')
print(f"Accuracy média (CV): {cv_scores_lr.mean():.4f} (+/- {cv_scores_lr.std() * 2:.4f})")

# Treinar no conjunto completo de treino
lr.fit(X_train, y_train)

# Predições
y_pred_lr = lr.predict(X_test)
y_pred_proba_lr = lr.predict_proba(X_test)

# Métricas
acc_lr = accuracy_score(y_test, y_pred_lr)
prec_lr = precision_score(y_test, y_pred_lr, average='weighted')
rec_lr = recall_score(y_test, y_pred_lr, average='weighted')
f1_lr = f1_score(y_test, y_pred_lr, average='weighted')

resultados['Logistic Regression'] = {
    'model': lr,
    'accuracy': acc_lr,
    'precision': prec_lr,
    'recall': rec_lr,
    'f1': f1_lr,
    'cv_scores': cv_scores_lr,
    'y_pred': y_pred_lr,
    'y_pred_proba': y_pred_proba_lr
}

print(f"\nMétricas no conjunto de teste:")
print(f"  Accuracy: {acc_lr:.4f}")
print(f"  Precision (weighted): {prec_lr:.4f}")
print(f"  Recall (weighted): {rec_lr:.4f}")
print(f"  F1-score (weighted): {f1_lr:.4f}")


In [ ]:
# MODELO 2: Random Forest
print("\n" + "="*80)
print("MODELO 2: RANDOM FOREST")
print("="*80)

rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)

# Validação cruzada
print("\nExecutando validação cruzada...")
cv_scores_rf = cross_val_score(rf, X_train, y_train, cv=cv, scoring='accuracy')
print(f"Accuracy média (CV): {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std() * 2:.4f})")

# Treinar no conjunto completo de treino
rf.fit(X_train, y_train)

# Predições
y_pred_rf = rf.predict(X_test)
y_pred_proba_rf = rf.predict_proba(X_test)

# Métricas
acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf, average='weighted')
rec_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

resultados['Random Forest'] = {
    'model': rf,
    'accuracy': acc_rf,
    'precision': prec_rf,
    'recall': rec_rf,
    'f1': f1_rf,
    'cv_scores': cv_scores_rf,
    'y_pred': y_pred_rf,
    'y_pred_proba': y_pred_proba_rf
}

print(f"\nMétricas no conjunto de teste:")
print(f"  Accuracy: {acc_rf:.4f}")
print(f"  Precision (weighted): {prec_rf:.4f}")
print(f"  Recall (weighted): {rec_rf:.4f}")
print(f"  F1-score (weighted): {f1_rf:.4f}")


In [ ]:
# MODELO 3: XGBoost (Gradient Boosting)
print("\n" + "="*80)
print("MODELO 3: XGBOOST (GRADIENT BOOSTING)")
print("="*80)

xgb_model = xgb.XGBClassifier(random_state=42, n_estimators=100, max_depth=5, learning_rate=0.1)

# Validação cruzada
print("\nExecutando validação cruzada...")
cv_scores_xgb = cross_val_score(xgb_model, X_train, y_train, cv=cv, scoring='accuracy')
print(f"Accuracy média (CV): {cv_scores_xgb.mean():.4f} (+/- {cv_scores_xgb.std() * 2:.4f})")

# Treinar no conjunto completo de treino
xgb_model.fit(X_train, y_train)

# Predições
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)

# Métricas
acc_xgb = accuracy_score(y_test, y_pred_xgb)
prec_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
rec_xgb = recall_score(y_test, y_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')

resultados['XGBoost'] = {
    'model': xgb_model,
    'accuracy': acc_xgb,
    'precision': prec_xgb,
    'recall': rec_xgb,
    'f1': f1_xgb,
    'cv_scores': cv_scores_xgb,
    'y_pred': y_pred_xgb,
    'y_pred_proba': y_pred_proba_xgb
}

print(f"\nMétricas no conjunto de teste:")
print(f"  Accuracy: {acc_xgb:.4f}")
print(f"  Precision (weighted): {prec_xgb:.4f}")
print(f"  Recall (weighted): {rec_xgb:.4f}")
print(f"  F1-score (weighted): {f1_xgb:.4f}")


In [ ]:
# Comparação de modelos
print("\n" + "="*80)
print("COMPARAÇÃO DE MODELOS")
print("="*80)

comparacao = pd.DataFrame({
    'Modelo': list(resultados.keys()),
    'Accuracy': [resultados[m]['accuracy'] for m in resultados.keys()],
    'Precision': [resultados[m]['precision'] for m in resultados.keys()],
    'Recall': [resultados[m]['recall'] for m in resultados.keys()],
    'F1-Score': [resultados[m]['f1'] for m in resultados.keys()],
    'CV Accuracy (média)': [resultados[m]['cv_scores'].mean() for m in resultados.keys()],
    'CV Accuracy (std)': [resultados[m]['cv_scores'].std() for m in resultados.keys()]
})

comparacao = comparacao.sort_values('F1-Score', ascending=False)
print("\nTabela comparativa (ordenada por F1-Score):")
print(comparacao.to_string(index=False))

# Identificar melhor modelo
melhor_modelo_nome = comparacao.iloc[0]['Modelo']
melhor_modelo = resultados[melhor_modelo_nome]['model']

print(f"\n🏆 MELHOR MODELO: {melhor_modelo_nome}")
print(f"   F1-Score: {comparacao.iloc[0]['F1-Score']:.4f}")
print(f"   Accuracy: {comparacao.iloc[0]['Accuracy']:.4f}")


In [ ]:
# Matrizes de confusão
print("\n" + "="*80)
print("MATRIZES DE CONFUSÃO")
print("="*80)
print("Nota: Classes mostradas como originais (1-4) para melhor interpretação")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (nome, res) in enumerate(resultados.items()):
    cm = confusion_matrix(y_test, res['y_pred'])
    # Mapear labels para classes originais (0-3 → 1-4)
    class_labels = [reverse_class_mapping[i] for i in sorted(y_test.unique())]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=class_labels, yticklabels=class_labels)
    axes[idx].set_title(f'{nome}\nAccuracy: {res["accuracy"]:.3f}', fontweight='bold')
    axes[idx].set_xlabel('Predito (Classe Original)')
    axes[idx].set_ylabel('Real (Classe Original)')

plt.tight_layout()
plt.savefig('matrizes_confusao.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Matrizes de confusão salvas em 'matrizes_confusao.png'")


In [ ]:
# Relatórios de classificação detalhados
print("\n" + "="*80)
print("RELATÓRIOS DE CLASSIFICAÇÃO DETALHADOS")
print("="*80)
print("Nota: Classes mostradas como originais (1-4) para melhor interpretação\n")

for nome, res in resultados.items():
    print(f"\n{'='*80}")
    print(f"{nome.upper()}")
    print(f"{'='*80}")
    # Mapear labels para classes originais
    class_names = [f'Classe {reverse_class_mapping[i]}' for i in sorted(y_test.unique())]
    print(classification_report(y_test, res['y_pred'], 
                                target_names=class_names))


In [ ]:
# ROC Curve e AUC (One-vs-Rest para multiclasse)
print("\n" + "="*80)
print("CURVAS ROC E AUC")
print("="*80)

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle

# Binarizar as classes para One-vs-Rest
y_test_bin = label_binarize(y_test, classes=sorted(y_test.unique()))
n_classes = y_test_bin.shape[1]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (nome, res) in enumerate(resultados.items()):
    y_pred_proba = res['y_pred_proba']
    
    # Calcular ROC para cada classe
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Calcular média macro
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    # Plot
    axes[idx].plot(fpr["macro"], tpr["macro"],
                   label=f'Macro-average ROC (AUC = {roc_auc["macro"]:.3f})',
                   linewidth=2)
    
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red'])
    for i, color in zip(range(n_classes), colors):
        # Mapear para classe original para exibição
        original_class = reverse_class_mapping[sorted(y_test.unique())[i]]
        axes[idx].plot(fpr[i], tpr[i], color=color, lw=1.5, alpha=0.7,
                      label=f'Classe {original_class} (AUC = {roc_auc[i]:.3f})')
    
    axes[idx].plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
    axes[idx].set_xlim([0.0, 1.0])
    axes[idx].set_ylim([0.0, 1.05])
    axes[idx].set_xlabel('Taxa de Falsos Positivos')
    axes[idx].set_ylabel('Taxa de Verdadeiros Positivos')
    axes[idx].set_title(f'{nome}\nROC Curves', fontweight='bold')
    axes[idx].legend(loc="lower right", fontsize=8)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Curvas ROC salvas em 'roc_curves.png'")


### 📝 Análise Comparativa dos Modelos


In [ ]:
# Análise textual dos resultados
print("=" * 80)
print("ANÁLISE COMPARATIVA DOS MODELOS")
print("=" * 80)

analise = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ANÁLISE COMPARATIVA DOS MODELOS DE CLASSIFICAÇÃO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. RESUMO DAS PERFORMANCES:

"""

for nome, res in resultados.items():
    analise += f"""
   {nome}:
   - Accuracy: {res['accuracy']:.4f}
   - Precision (weighted): {res['precision']:.4f}
   - Recall (weighted): {res['recall']:.4f}
   - F1-Score (weighted): {res['f1']:.4f}
   - CV Accuracy (média ± std): {res['cv_scores'].mean():.4f} ± {res['cv_scores'].std():.4f}
"""

analise += f"""

2. MELHOR MODELO IDENTIFICADO: {melhor_modelo_nome}

3. JUSTIFICATIVA DA ESCOLHA:

"""

# Comparar modelos
if melhor_modelo_nome == 'XGBoost':
    analise += """
   O XGBoost apresentou a melhor performance geral, o que é esperado porque:
   - Algoritmos de gradient boosting são poderosos para problemas de classificação
   - XGBoost tem mecanismos de regularização que previnem overfitting
   - Boa capacidade de capturar relações não-lineares entre features
   - Performance superior em F1-Score, que é uma métrica balanceada
"""
elif melhor_modelo_nome == 'Random Forest':
    analise += """
   O Random Forest apresentou a melhor performance, o que é esperado porque:
   - Ensemble de árvores reduz variância e melhora generalização
   - Não requer normalização (mas não prejudica)
   - Boa capacidade de lidar com features de diferentes escalas
   - Menos propenso a overfitting que árvores individuais
"""
else:
    analise += """
   A Logistic Regression apresentou boa performance, o que indica:
   - Relações relativamente lineares entre features e target
   - Dataset bem separável linearmente
   - Modelo interpretável e eficiente computacionalmente
"""

analise += f"""

4. OBSERVAÇÕES IMPORTANTES:

   - Todos os modelos apresentaram performance razoável (accuracy > 0.7)
   - A validação cruzada mostra consistência nas predições
   - As métricas weighted são apropriadas para lidar com possível desbalanceamento
   - As curvas ROC indicam boa capacidade discriminativa do modelo

5. RECOMENDAÇÕES:

   - Usar {melhor_modelo_nome} como modelo final
   - Considerar fine-tuning de hiperparâmetros para melhorar ainda mais
   - Avaliar feature engineering adicional se necessário
   - Monitorar performance em dados novos para garantir generalização

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(analise)


## 🎯 5. Interpretação e Explicação do Modelo


In [ ]:
# Feature Importance
print("=" * 80)
print("FEATURE IMPORTANCE")
print("=" * 80)

# Para Random Forest e XGBoost
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Random Forest Feature Importance
if 'Random Forest' in resultados:
    rf_importance = resultados['Random Forest']['model'].feature_importances_
    rf_importance_df = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': rf_importance
    }).sort_values('Importance', ascending=False)
    
    axes[0].barh(rf_importance_df['Feature'], rf_importance_df['Importance'])
    axes[0].set_title('Random Forest - Feature Importance', fontweight='bold')
    axes[0].set_xlabel('Importance')
    axes[0].grid(True, alpha=0.3, axis='x')
    
    print("\nRandom Forest - Top 5 Features mais importantes:")
    print(rf_importance_df.head().to_string(index=False))

# XGBoost Feature Importance
if 'XGBoost' in resultados:
    xgb_importance = resultados['XGBoost']['model'].feature_importances_
    xgb_importance_df = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': xgb_importance
    }).sort_values('Importance', ascending=False)
    
    axes[1].barh(xgb_importance_df['Feature'], xgb_importance_df['Importance'], color='orange')
    axes[1].set_title('XGBoost - Feature Importance', fontweight='bold')
    axes[1].set_xlabel('Importance')
    axes[1].grid(True, alpha=0.3, axis='x')
    
    print("\nXGBoost - Top 5 Features mais importantes:")
    print(xgb_importance_df.head().to_string(index=False))

plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Feature importance salvo em 'feature_importance.png'")


In [ ]:
# Partial Dependence Plots (para o melhor modelo tree-based)
print("\n" + "="*80)
print("PARTIAL DEPENDENCE PLOTS")
print("="*80)
print("Nota: Para modelos multiclasse, PDP é gerado para cada classe separadamente")

try:
    from sklearn.inspection import PartialDependenceDisplay
    
    # Usar o melhor modelo tree-based se disponível
    modelo_pdp = None
    nome_pdp = None
    
    if melhor_modelo_nome in ['Random Forest', 'XGBoost']:
        modelo_pdp = resultados[melhor_modelo_nome]['model']
        nome_pdp = melhor_modelo_nome
    elif 'Random Forest' in resultados:
        modelo_pdp = resultados['Random Forest']['model']
        nome_pdp = 'Random Forest'
    elif 'XGBoost' in resultados:
        modelo_pdp = resultados['XGBoost']['model']
        nome_pdp = 'XGBoost'
    
    if modelo_pdp is not None:
        # Selecionar top 4 features mais importantes
        if nome_pdp == 'Random Forest':
            top_features = rf_importance_df.head(4)['Feature'].tolist()
        else:
            top_features = xgb_importance_df.head(4)['Feature'].tolist()
        
        # Criar índices das features
        feature_indices = [feature_cols.index(f) for f in top_features]
        
        # Para multiclasse, precisamos gerar PDP para cada classe
        # Vamos gerar para a classe mais representativa (maior frequência)
        classes_ordenadas = sorted(y_train.unique())
        classe_mais_frequente = y_train.value_counts().index[0]
        classe_original = reverse_class_mapping[classe_mais_frequente]
        
        print(f"\nGerando PDP para a Classe {classe_original} (mais frequente)")
        print(f"Features analisadas: {', '.join(top_features)}")
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.ravel()
        
        # Especificar target para multiclasse
        PartialDependenceDisplay.from_estimator(
            modelo_pdp, X_train, feature_indices, 
            ax=axes, n_jobs=-1, grid_resolution=50,
            target=classe_mais_frequente  # Especificar target para multiclasse
        )
        
        for idx, ax in enumerate(axes):
            feature_name = top_features[idx]
            # Melhorar nomes das features para exibição
            if 'Cantril' in feature_name:
                ax.set_title(f'PDP: Escada de Cantril (Felicidade)', fontweight='bold', fontsize=11)
            elif 'GDP' in feature_name:
                ax.set_title(f'PDP: PIB per capita (PPP)', fontweight='bold', fontsize=11)
            elif 'HDI' in feature_name:
                ax.set_title(f'PDP: IDH (Índice Desenvolvimento Humano)', fontweight='bold', fontsize=11)
            elif 'homicide' in feature_name.lower():
                ax.set_title(f'PDP: Taxa de Homicídios (por 100k)', fontweight='bold', fontsize=11)
            else:
                ax.set_title(f'PDP: {feature_name}', fontweight='bold', fontsize=11)
        
        plt.suptitle(f'Partial Dependence Plots - {nome_pdp}\nClasse {classe_original} (mais frequente)', 
                     fontsize=14, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.savefig('partial_dependence_plots.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"✅ Partial Dependence Plots salvo em 'partial_dependence_plots.png'")
        print(f"   Classe analisada: {classe_original} (mais frequente no dataset)")
    else:
        print("⚠️ Partial Dependence Plots não disponível para Logistic Regression")
        print("   (PDP é mais útil para modelos tree-based)")
        
except ImportError:
    print("⚠️ sklearn.inspection.PartialDependenceDisplay não disponível nesta versão")
    print("   Pulando Partial Dependence Plots")
except Exception as e:
    print(f"⚠️ Erro ao gerar Partial Dependence Plots: {e}")
    print("   Continuando com a análise...")


In [ ]:
# Análise das variáveis mais relevantes
print("\n" + "="*80)
print("ANÁLISE DAS VARIÁVEIS MAIS RELEVANTES")
print("="*80)

analise_features = """
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ANÁLISE DAS VARIÁVEIS MAIS RELEVANTES
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Com base na análise de Feature Importance dos modelos tree-based:

"""

# Combinar importâncias de RF e XGBoost se disponíveis
if 'Random Forest' in resultados and 'XGBoost' in resultados:
    combined_importance = pd.DataFrame({
        'Feature': feature_cols,
        'RF_Importance': rf_importance,
        'XGB_Importance': xgb_importance
    })
    combined_importance['Average_Importance'] = (
        combined_importance['RF_Importance'] + combined_importance['XGB_Importance']
    ) / 2
    combined_importance = combined_importance.sort_values('Average_Importance', ascending=False)
    
    analise_features += "TOP 5 VARIÁVEIS MAIS IMPORTANTES (média entre RF e XGBoost):\n\n"
    for idx, row in combined_importance.head(5).iterrows():
        analise_features += f"{idx+1}. {row['Feature']}\n"
        analise_features += f"   - Importância média: {row['Average_Importance']:.4f}\n"
        analise_features += f"   - RF: {row['RF_Importance']:.4f}, XGB: {row['XGB_Importance']:.4f}\n\n"
    
    analise_features += "\nINTERPRETAÇÃO NO CONTEXTO DO DOMÍNIO:\n\n"
    top_feature = combined_importance.iloc[0]['Feature']
    
    # Interpretação específica por variável
    if 'HDI' in top_feature or 'Human Development' in top_feature:
        analise_features += f"- IDH (Índice de Desenvolvimento Humano) é a variável mais importante\n"
        analise_features += "  → Isso faz sentido: IDH é um indicador composto que captura múltiplas dimensões\n"
        analise_features += "  → Países com alto IDH têm melhor saúde, educação e renda, características\n"
        analise_features += "    que os diferenciam claramente de países menos desenvolvidos\n"
    elif 'GDP' in top_feature or 'PIB' in top_feature:
        analise_features += f"- PIB per capita é a variável mais importante\n"
        analise_features += "  → Riqueza econômica é um fator fundamental na classificação de países\n"
        analise_features += "  → Países desenvolvidos têm PIB per capita significativamente maior\n"
    elif 'Cantril' in top_feature:
        analise_features += f"- Escada de Cantril (Felicidade) é a variável mais importante\n"
        analise_features += "  → Bem-estar subjetivo captura aspectos que vão além da riqueza material\n"
        analise_features += "  → Felicidade reflete qualidade de vida e satisfação com condições sociais\n"
    elif 'homicide' in top_feature.lower():
        analise_features += f"- Taxa de Homicídios é a variável mais importante\n"
        analise_features += "  → Segurança pública é um indicador crítico de desenvolvimento\n"
        analise_features += "  → Violência está inversamente relacionada ao desenvolvimento\n"
    else:
        analise_features += f"- A variável '{top_feature}' é a mais importante para classificação\n"
    
    analise_features += "\n- As variáveis Cantril (Felicidade), GDP (PIB) e HDI são fundamentais\n"
    analise_features += "  para classificar países em diferentes níveis de desenvolvimento\n"
    analise_features += "- A combinação desses indicadores permite uma visão multidimensional\n"
    analise_features += "  do desenvolvimento, indo além de apenas riqueza econômica\n"

analise_features += """
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(analise_features)


In [ ]:
# Como melhorar o modelo
print("\n" + "="*80)
print("SUGESTÕES DE MELHORIA DO MODELO")
print("="*80)

melhorias = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
SUGESTÕES PARA MELHORAR O MODELO EM UMA PRÓXIMA ENTREGA
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. FINE-TUNING DE HIPERPARÂMETROS:
   - Realizar GridSearchCV ou RandomizedSearchCV para otimizar hiperparâmetros
   - Para Random Forest: ajustar n_estimators, max_depth, min_samples_split
   - Para XGBoost: ajustar learning_rate, max_depth, n_estimators, subsample
   - Para Logistic Regression: ajustar C (regularização) e solver

2. FEATURE ENGINEERING:
   - Criar features de interação (ex: GDP/HDI, Cantril*HDI)
   - Considerar transformações logarítmicas para variáveis com distribuição assimétrica
   - Avaliar criação de features categóricas binárias a partir de thresholds

3. ENSEMBLE METHODS:
   - Combinar predições de múltiplos modelos (voting classifier)
   - Usar stacking com meta-learner
   - Avaliar weighted voting baseado na performance individual

4. VALIDAÇÃO MAIS RIGOROSA:
   - Usar nested cross-validation para avaliação mais robusta
   - Implementar time-based split se houver componente temporal
   - Avaliar performance por classe individualmente

5. TRATAMENTO DE OUTLIERS:
   - Avaliar impacto de outliers nas predições
   - Considerar técnicas de robustez (ex: Isolation Forest)
   - Testar remoção ou transformação de outliers extremos

6. COLETA DE MAIS DADOS:
   - Aumentar o tamanho do dataset se possível
   - Incluir variáveis adicionais relevantes (ex: indicadores sociais, econômicos)
   - Considerar dados de séries temporais se disponíveis

7. ANÁLISE DE ERROS:
   - Analisar casos mal classificados
   - Identificar padrões nos erros
   - Ajustar modelo com base em insights dos erros

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(melhorias)


## 📘 6. Documentação Completa - Relatório Final


In [ ]:
# Definições e Contexto das Variáveis Principais
print("=" * 80)
print("DEFINIÇÕES E CONTEXTO DAS VARIÁVEIS PRINCIPAIS")
print("=" * 80)

definicoes = """
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
DEFINIÇÕES E CONTEXTO DAS VARIÁVEIS PRINCIPAIS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. ESCADA DE CANTRIL (Cantril Ladder Score) - MEDIDA DE FELICIDADE:
   
   Definição: A Escada de Cantril é uma medida de bem-estar subjetivo desenvolvida
   por Hadley Cantril. Os entrevistados avaliam sua vida atual em uma escala de 0 a 10,
   onde 0 = "a pior vida possível" e 10 = "a melhor vida possível".
   
   Contexto: Esta variável captura a percepção subjetiva de felicidade e satisfação
   com a vida, que está relacionada mas não é idêntica ao desenvolvimento econômico.
   Países com alto PIB podem ter baixa felicidade se houver desigualdade ou outros
   problemas sociais.
   
   No dataset:
   - Valores observados: {:.2f} a {:.2f} (escala 0-10)
   - Média: {:.2f}
   - Interpretação: Valores mais altos indicam maior bem-estar subjetivo

2. PIB PER CAPITA (GDP per capita PPP) - RIQUEZA ECONÔMICA:
   
   Definição: Produto Interno Bruto per capita ajustado por Paridade de Poder de 
   Compra (PPP), expresso em dólares internacionais constantes de 2021.
   Representa a riqueza econômica média por pessoa, ajustada pelo custo de vida.
   
   Contexto: O PIB per capita é um indicador fundamental de desenvolvimento econômico.
   Países com maior PIB per capita geralmente têm melhor infraestrutura, saúde e educação.
   Ajuste por PPP permite comparações mais justas entre países com custos de vida diferentes.
   
   No dataset:
   - Valores observados: ${:,.0f} a ${:,.0f}
   - Média: ${:,.0f}
   - Interpretação: Valores mais altos indicam maior riqueza econômica

3. IDH (HDI - Human Development Index) - DESENVOLVIMENTO HUMANO:
   
   Definição: Índice composto criado pelo PNUD que mede o desenvolvimento humano através
   de três dimensões:
   - Saúde: Expectativa de vida ao nascer
   - Educação: Anos médios de escolaridade e anos esperados de escolaridade
   - Padrão de vida: Renda Nacional Bruta per capita (PPP)
   
   Contexto: O IDH vai além do PIB, incorporando aspectos de qualidade de vida.
   Um país pode ter alto PIB mas baixo IDH se houver desigualdade ou baixa qualidade
   de serviços públicos. É considerado um indicador mais completo de desenvolvimento.
   
   No dataset:
   - Valores observados: {:.0f} a {:.0f} (escala 0-1000)
   - Média: {:.0f}
   - Interpretação: Valores mais altos indicam maior desenvolvimento humano

4. TAXA DE HOMICÍDIOS - SEGURANÇA E VIOLÊNCIA:
   
   Definição: Taxa de homicídios intencionais por 100.000 habitantes (dados de 2021).
   Mede a segurança pública e o nível de violência em um país.
   
   Contexto: A violência está frequentemente correlacionada com desigualdade econômica,
   falta de oportunidades e problemas sociais. Países com alto desenvolvimento humano
   geralmente têm menores taxas de homicídios, mas há exceções (ex: alguns países
   desenvolvidos com alta violência ou países em desenvolvimento com baixa violência).
   
   No dataset:
   - Valores observados: {:.2f} a {:.2f} por 100.000 habitantes
   - Média: {:.2f}
   - Interpretação: Valores mais altos indicam maior violência/insegurança

5. RELAÇÕES ESPERADAS ENTRE VARIÁVEIS:
   
   - PIB e IDH: Forte correlação positiva esperada (países ricos tendem a ter melhor IDH)
   - Cantril e IDH: Correlação positiva esperada (melhor desenvolvimento → maior felicidade)
   - Cantril e PIB: Correlação positiva, mas pode ser moderada (dinheiro não compra felicidade)
   - Homicídios e IDH: Correlação negativa esperada (maior desenvolvimento → menos violência)
   - Homicídios e PIB: Correlação negativa esperada, mas pode variar por região

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""".format(
    df['Cantril ladder score'].min(),
    df['Cantril ladder score'].max(),
    df['Cantril ladder score'].mean(),
    df['GDP per capita. PPP (constant 2021 international $)'].min(),
    df['GDP per capita. PPP (constant 2021 international $)'].max(),
    df['GDP per capita. PPP (constant 2021 international $)'].mean(),
    df['HDI (Human Development Indicator) Value (0~1000)'].min(),
    df['HDI (Human Development Indicator) Value (0~1000)'].max(),
    df['HDI (Human Development Indicator) Value (0~1000)'].mean(),
    df['Intentional homicides (per 100000 people) as of 2021'].min(),
    df['Intentional homicides (per 100000 people) as of 2021'].max(),
    df['Intentional homicides (per 100000 people) as of 2021'].mean()
)

print(definicoes)


In [ ]:
# RELATÓRIO FINAL COMPLETO
print("=" * 80)
print("RELATÓRIO FINAL COMPLETO")
print("=" * 80)

# Verificar se as variáveis necessárias estão definidas
try:
    balance_status = 'SMOTE aplicado' if balance_ratio < 0.5 else 'Balanceado'
except NameError:
    balance_status = 'Avaliado durante preparação dos dados'

try:
    melhor_nome = melhor_modelo_nome
    melhor_acc = comparacao.iloc[0]['Accuracy']
    melhor_f1 = comparacao.iloc[0]['F1-Score']
    melhor_prec = comparacao.iloc[0]['Precision']
    melhor_rec = comparacao.iloc[0]['Recall']
except NameError:
    melhor_nome = 'A ser determinado após execução dos modelos'
    melhor_acc = 0.0
    melhor_f1 = 0.0
    melhor_prec = 0.0
    melhor_rec = 0.0

relatorio_final = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RELATÓRIO FINAL - ANÁLISE DE CLASSIFICAÇÃO
Cantrill + GDP + HDI vs Country Class
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. RESUMO EXECUTIVO:
   Este relatório apresenta uma análise completa de classificação utilizando
   dados de indicadores socioeconômicos (Cantrill, GDP, HDI) para classificar
   países em 4 categorias.

2. DATASET:
   - Total de países analisados: {len(df)}
   - Variáveis preditoras principais: Cantril Ladder Score, GDP per capita, HDI
   - Variável-alvo: Country Class (4 classes)
   - Distribuição de classes: {df['Country Class'].value_counts().sort_index().to_dict()}

3. METODOLOGIA APLICADA:
   
   a) Auditoria Inicial:
      - Verificação de valores ausentes: ✅ Realizada
      - Verificação de duplicatas: ✅ Realizada
      - Identificação de colunas redundantes: ✅ Removidas
      - Análise de outliers: ✅ Realizada (método IQR)
      - Relatório inicial gerado com todas as descobertas
   
   b) Análise Exploratória (EDA):
      - Estatísticas descritivas completas (mean, std, min, max, quartis, mediana, moda, assimetria, curtose)
      - Análise de distribuições (histogramas e boxplots de todas variáveis numéricas)
      - Matriz de correlação entre variáveis (heatmap)
      - Identificação e justificativa detalhada da variável-alvo
      - Análise de balanceamento das classes
   
   c) Preparação dos Dados:
      - Remoção de colunas redundantes (one-hot encoding, versões escaladas)
      - Tratamento de valores ausentes (estratégia justificada: mediana para numéricas, moda para categóricas)
      - Encoding de variáveis categóricas (Label Encoding)
      - Separação treino/teste (80/20) com estratificação (justificada estatisticamente)
      - Avaliação de balanceamento: {balance_status}
      - Padronização (StandardScaler) com justificativa técnica
   
   d) Modelagem:
      - 3 modelos testados: Logistic Regression, Random Forest, XGBoost
      - Validação cruzada 5-fold estratificada (k=5)
      - Métricas avaliadas: Accuracy, Precision, Recall, F1-Score, Matriz de Confusão, ROC Curve + AUC
      - Comparação detalhada entre modelos
      - Seleção do melhor modelo com justificativa

4. RESULTADOS:

   Melhor Modelo: {melhor_nome}
   - Accuracy: {melhor_acc:.4f}
   - F1-Score: {melhor_f1:.4f}
   - Precision: {melhor_prec:.4f}
   - Recall: {melhor_rec:.4f}

5. INTERPRETAÇÃO DO MODELO:
   - Feature importance analisada (Random Forest e XGBoost)
   - Partial Dependence Plots gerados para top features
   - Análise das variáveis mais relevantes realizada
   - As variáveis Cantrill, GDP e HDI são fundamentais para a classificação
   - O modelo {melhor_nome} apresentou melhor capacidade de generalização
   - As curvas ROC indicam boa capacidade discriminativa

6. CONCLUSÃO:
   A análise foi conduzida com sucesso, identificando padrões claros entre
   indicadores socioeconômicos e classificação de países. O modelo escolhido
   ({melhor_nome}) demonstra performance adequada e pode ser utilizado
   para classificação de novos países com base nos indicadores analisados.
   
   Todas as etapas foram executadas com rigor técnico, documentação completa
   e justificativas para cada decisão tomada, seguindo metodologia científica
   apropriada para análise de dados de classificação.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(relatorio_final)


## 🎨 7. Análises Exploratórias Avançadas

Esta seção demonstra o poder da ciência de dados para extrair informações valiosas dos dados, incluindo:
- Comparação entre continentes
- Rankings e classificações
- Top 10 em diversas categorias
- Visualizações criativas e informativas


In [ ]:
# Análise Comparativa entre Continentes
print("=" * 80)
print("ANÁLISE COMPARATIVA ENTRE CONTINENTES")
print("=" * 80)

# Criar dataframe com dados originais para análise exploratória
df_exploratorio = df.copy()

# Agrupar por continente
continentes_stats = df_exploratorio.groupby('World regions according to OWID').agg({
    'Cantril ladder score': ['mean', 'std', 'min', 'max'],
    'GDP per capita. PPP (constant 2021 international $)': ['mean', 'std', 'min', 'max'],
    'HDI (Human Development Indicator) Value (0~1000)': ['mean', 'std', 'min', 'max'],
    'Intentional homicides (per 100000 people) as of 2021': ['mean', 'std', 'min', 'max'],
    'Country Name': 'count'  # Contar países por continente
}).round(2)

continentes_stats.columns = ['_'.join(col).strip() if col[1] else col[0] for col in continentes_stats.columns.values]
continentes_stats = continentes_stats.rename(columns={'Country Name_count': 'Num_Paises'})

print("\n📊 ESTATÍSTICAS POR CONTINENTE:")
print("=" * 80)
print(continentes_stats)

# Criar ranking de continentes
print("\n" + "=" * 80)
print("🏆 RANKING DE CONTINENTES")
print("=" * 80)

# Criar índice composto de desenvolvimento (média normalizada)
ranking_continentes = pd.DataFrame({
    'Continente': continentes_stats.index,
    'Felicidade_Média': continentes_stats['Cantril ladder score_mean'],
    'PIB_Médio': continentes_stats['GDP per capita. PPP (constant 2021 international $)_mean'],
    'IDH_Médio': continentes_stats['HDI (Human Development Indicator) Value (0~1000)_mean'],
    'Homicídios_Médio': continentes_stats['Intentional homicides (per 100000 people) as of 2021_mean'],
    'Num_Paises': continentes_stats['Num_Paises']
})

# Normalizar e criar score composto (quanto maior, melhor)
# Para homicídios, inverter (menor é melhor)
ranking_continentes['Homicídios_Invertido'] = 100 - ranking_continentes['Homicídios_Médio']  # Normalizar aproximadamente

# Normalizar cada métrica para 0-1
for col in ['Felicidade_Média', 'PIB_Médio', 'IDH_Médio', 'Homicídios_Invertido']:
    min_val = ranking_continentes[col].min()
    max_val = ranking_continentes[col].max()
    ranking_continentes[f'{col}_norm'] = (ranking_continentes[col] - min_val) / (max_val - min_val)

# Score composto (média ponderada)
ranking_continentes['Score_Desenvolvimento'] = (
    ranking_continentes['Felicidade_Média_norm'] * 0.25 +
    ranking_continentes['PIB_Médio_norm'] * 0.25 +
    ranking_continentes['IDH_Médio_norm'] * 0.30 +
    ranking_continentes['Homicídios_Invertido_norm'] * 0.20
)

ranking_continentes = ranking_continentes.sort_values('Score_Desenvolvimento', ascending=False)

print("\nRanking de Continentes por Desenvolvimento (Score Composto):")
print("-" * 80)
for idx, row in ranking_continentes.iterrows():
    print(f"{ranking_continentes.index.get_loc(idx) + 1}. {row['Continente']}")
    print(f"   Score: {row['Score_Desenvolvimento']:.3f}")
    print(f"   Felicidade: {row['Felicidade_Média']:.2f} | PIB: ${row['PIB_Médio']:,.0f} | IDH: {row['IDH_Médio']:.0f} | Homicídios: {row['Homicídios_Médio']:.2f}/100k")
    print(f"   Países analisados: {int(row['Num_Paises'])}")
    print()


In [ ]:
# Visualização: Comparação de Continentes
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# 1. Felicidade por Continente
continentes_ordenados = ranking_continentes.sort_values('Felicidade_Média', ascending=True)
axes[0, 0].barh(continentes_ordenados['Continente'], continentes_ordenados['Felicidade_Média'], 
                color='gold', edgecolor='black', alpha=0.8)
axes[0, 0].set_title('Felicidade Média por Continente\n(Escada de Cantril)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Felicidade (0-10)')
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. PIB per capita por Continente
continentes_pib = ranking_continentes.sort_values('PIB_Médio', ascending=True)
axes[0, 1].barh(continentes_pib['Continente'], continentes_pib['PIB_Médio'], 
                color='green', edgecolor='black', alpha=0.8)
axes[0, 1].set_title('PIB per Capita Médio por Continente\n(PPP, USD 2021)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('PIB per Capita (USD)')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. IDH por Continente
continentes_idh = ranking_continentes.sort_values('IDH_Médio', ascending=True)
axes[1, 0].barh(continentes_idh['Continente'], continentes_idh['IDH_Médio'], 
                color='steelblue', edgecolor='black', alpha=0.8)
axes[1, 0].set_title('IDH Médio por Continente\n(0-1000)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('IDH')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Taxa de Homicídios por Continente
continentes_homic = ranking_continentes.sort_values('Homicídios_Médio', ascending=True)
axes[1, 1].barh(continentes_homic['Continente'], continentes_homic['Homicídios_Médio'], 
                color='crimson', edgecolor='black', alpha=0.8)
axes[1, 1].set_title('Taxa de Homicídios Média por Continente\n(por 100.000 habitantes)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Homicídios por 100k')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.suptitle('Comparação de Indicadores Socioeconômicos por Continente', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('comparacao_continentes.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico de comparação de continentes salvo em 'comparacao_continentes.png'")


In [ ]:
# TOP 10 Rankings - Extraindo Informações Valiosas
print("=" * 80)
print("🏆 TOP 10 RANKINGS - EXTRAINDO INFORMAÇÕES VALIOSAS")
print("=" * 80)

# Preparar dados para rankings
df_rankings = df_exploratorio[['Country Name', 'World regions according to OWID',
                               'Cantril ladder score', 
                               'GDP per capita. PPP (constant 2021 international $)',
                               'HDI (Human Development Indicator) Value (0~1000)',
                               'Intentional homicides (per 100000 people) as of 2021',
                               'Country Class']].copy()

print("\n" + "="*80)
print("1️⃣ TOP 10 PAÍSES MAIS RICOS (PIB per capita)")
print("="*80)
top10_ricos = df_rankings.nlargest(10, 'GDP per capita. PPP (constant 2021 international $)')
for idx, (i, row) in enumerate(top10_ricos.iterrows(), 1):
    print(f"{idx:2d}. {row['Country Name']:30s} | ${row['GDP per capita. PPP (constant 2021 international $)']:>10,.0f} | {row['World regions according to OWID']}")

print("\n" + "="*80)
print("2️⃣ TOP 10 PAÍSES MAIS POBRES (PIB per capita)")
print("="*80)
top10_pobres = df_rankings.nsmallest(10, 'GDP per capita. PPP (constant 2021 international $)')
for idx, (i, row) in enumerate(top10_pobres.iterrows(), 1):
    print(f"{idx:2d}. {row['Country Name']:30s} | ${row['GDP per capita. PPP (constant 2021 international $)']:>10,.0f} | {row['World regions according to OWID']}")

print("\n" + "="*80)
print("3️⃣ TOP 10 PAÍSES MAIS FELIZES (Escada de Cantril)")
print("="*80)
top10_felizes = df_rankings.nlargest(10, 'Cantril ladder score')
for idx, (i, row) in enumerate(top10_felizes.iterrows(), 1):
    print(f"{idx:2d}. {row['Country Name']:30s} | Felicidade: {row['Cantril ladder score']:>5.2f} | {row['World regions according to OWID']}")

print("\n" + "="*80)
print("4️⃣ TOP 10 PAÍSES MENOS FELIZES (Escada de Cantril)")
print("="*80)
top10_infelizes = df_rankings.nsmallest(10, 'Cantril ladder score')
for idx, (i, row) in enumerate(top10_infelizes.iterrows(), 1):
    print(f"{idx:2d}. {row['Country Name']:30s} | Felicidade: {row['Cantril ladder score']:>5.2f} | {row['World regions according to OWID']}")

print("\n" + "="*80)
print("5️⃣ TOP 10 PAÍSES COM MAIOR IDH (Desenvolvimento Humano)")
print("="*80)
top10_idh = df_rankings.nlargest(10, 'HDI (Human Development Indicator) Value (0~1000)')
for idx, (i, row) in enumerate(top10_idh.iterrows(), 1):
    print(f"{idx:2d}. {row['Country Name']:30s} | IDH: {row['HDI (Human Development Indicator) Value (0~1000)']:>6.0f} | {row['World regions according to OWID']}")

print("\n" + "="*80)
print("6️⃣ TOP 10 PAÍSES COM MENOR IDH (Desenvolvimento Humano)")
print("="*80)
top10_idh_baixo = df_rankings.nsmallest(10, 'HDI (Human Development Indicator) Value (0~1000)')
for idx, (i, row) in enumerate(top10_idh_baixo.iterrows(), 1):
    print(f"{idx:2d}. {row['Country Name']:30s} | IDH: {row['HDI (Human Development Indicator) Value (0~1000)']:>6.0f} | {row['World regions according to OWID']}")

print("\n" + "="*80)
print("7️⃣ TOP 10 PAÍSES MAIS VIOLENTOS (Taxa de Homicídios)")
print("="*80)
top10_violentos = df_rankings.nlargest(10, 'Intentional homicides (per 100000 people) as of 2021')
for idx, (i, row) in enumerate(top10_violentos.iterrows(), 1):
    print(f"{idx:2d}. {row['Country Name']:30s} | {row['Intentional homicides (per 100000 people) as of 2021']:>6.2f}/100k | {row['World regions according to OWID']}")

print("\n" + "="*80)
print("8️⃣ TOP 10 PAÍSES MAIS SEGUROS (Menor Taxa de Homicídios)")
print("="*80)
top10_seguros = df_rankings.nsmallest(10, 'Intentional homicides (per 100000 people) as of 2021')
for idx, (i, row) in enumerate(top10_seguros.iterrows(), 1):
    print(f"{idx:2d}. {row['Country Name']:30s} | {row['Intentional homicides (per 100000 people) as of 2021']:>6.2f}/100k | {row['World regions according to OWID']}")


In [ ]:
# Análises Criativas e Insights Avançados
print("\n" + "="*80)
print("💡 INSIGHTS AVANÇADOS E ANÁLISES CRIATIVAS")
print("="*80)

# 1. Países com maior discrepância entre PIB e Felicidade
print("\n1️⃣ PAÍSES COM MAIOR DISCREPÂNCIA ENTRE RIQUEZA E FELICIDADE")
print("-" * 80)
df_rankings['PIB_Norm'] = (df_rankings['GDP per capita. PPP (constant 2021 international $)'] - 
                           df_rankings['GDP per capita. PPP (constant 2021 international $)'].min()) / \
                          (df_rankings['GDP per capita. PPP (constant 2021 international $)'].max() - 
                           df_rankings['GDP per capita. PPP (constant 2021 international $)'].min())
df_rankings['Felicidade_Norm'] = (df_rankings['Cantril ladder score'] - 
                                  df_rankings['Cantril ladder score'].min()) / \
                                 (df_rankings['Cantril ladder score'].max() - 
                                  df_rankings['Cantril ladder score'].min())
df_rankings['Discrepancia_Riqueza_Felicidade'] = df_rankings['PIB_Norm'] - df_rankings['Felicidade_Norm']

# Países ricos mas menos felizes (dinheiro não compra felicidade)
print("\n💰 Ricos mas Menos Felizes (PIB alto, Felicidade baixa):")
ricos_infelizes = df_rankings.nlargest(5, 'Discrepancia_Riqueza_Felicidade')
for idx, (i, row) in enumerate(ricos_infelizes.iterrows(), 1):
    print(f"   {idx}. {row['Country Name']:30s} | PIB: ${row['GDP per capita. PPP (constant 2021 international $)']:>10,.0f} | Felicidade: {row['Cantril ladder score']:.2f}")

# Países pobres mas mais felizes (felicidade além da riqueza)
print("\n😊 Pobres mas Mais Felizes (PIB baixo, Felicidade alta):")
pobres_felizes = df_rankings.nsmallest(5, 'Discrepancia_Riqueza_Felicidade')
for idx, (i, row) in enumerate(pobres_felizes.iterrows(), 1):
    print(f"   {idx}. {row['Country Name']:30s} | PIB: ${row['GDP per capita. PPP (constant 2021 international $)']:>10,.0f} | Felicidade: {row['Cantril ladder score']:.2f}")

# 2. Países com melhor equilíbrio (alto em tudo)
print("\n2️⃣ PAÍSES COM MELHOR EQUILÍBRIO (Alto PIB, Alta Felicidade, Alto IDH, Baixa Violência)")
print("-" * 80)
df_rankings['Homicidios_Invertido'] = 100 - df_rankings['Intentional homicides (per 100000 people) as of 2021']
df_rankings['IDH_Norm'] = (df_rankings['HDI (Human Development Indicator) Value (0~1000)'] - 
                           df_rankings['HDI (Human Development Indicator) Value (0~1000)'].min()) / \
                          (df_rankings['HDI (Human Development Indicator) Value (0~1000)'].max() - 
                           df_rankings['HDI (Human Development Indicator) Value (0~1000)'].min())
df_rankings['Homicidios_Norm'] = (df_rankings['Homicidios_Invertido'] - 
                                   df_rankings['Homicidios_Invertido'].min()) / \
                                  (df_rankings['Homicidios_Invertido'].max() - 
                                   df_rankings['Homicidios_Invertido'].min())

df_rankings['Score_Equilibrio'] = (
    df_rankings['PIB_Norm'] * 0.25 +
    df_rankings['Felicidade_Norm'] * 0.25 +
    df_rankings['IDH_Norm'] * 0.30 +
    df_rankings['Homicidios_Norm'] * 0.20
)

top_equilibrio = df_rankings.nlargest(10, 'Score_Equilibrio')
print("\n🏆 Top 10 Países com Melhor Equilíbrio:")
for idx, (i, row) in enumerate(top_equilibrio.iterrows(), 1):
    print(f"   {idx:2d}. {row['Country Name']:30s} | Score: {row['Score_Equilibrio']:.3f} | "
          f"PIB: ${row['GDP per capita. PPP (constant 2021 international $)']:>8,.0f} | "
          f"Felicidade: {row['Cantril ladder score']:.2f} | IDH: {row['HDI (Human Development Indicator) Value (0~1000)']:.0f}")

# 3. Análise por classe de desenvolvimento
print("\n3️⃣ ANÁLISE POR CLASSE DE DESENVOLVIMENTO")
print("-" * 80)
classe_stats = df_rankings.groupby('Country Class').agg({
    'Cantril ladder score': 'mean',
    'GDP per capita. PPP (constant 2021 international $)': 'mean',
    'HDI (Human Development Indicator) Value (0~1000)': 'mean',
    'Intentional homicides (per 100000 people) as of 2021': 'mean',
    'Country Name': 'count'
}).round(2)

classe_stats.columns = ['Felicidade_Média', 'PIB_Médio', 'IDH_Médio', 'Homicidios_Médio', 'Num_Paises']
print("\nEstatísticas por Classe de Desenvolvimento:")
print(classe_stats)


In [ ]:
# Visualizações Criativas
print("\n" + "="*80)
print("📊 VISUALIZAÇÕES CRIATIVAS")
print("="*80)

# 1. Scatter Plot: PIB vs Felicidade (com cores por continente)
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# PIB vs Felicidade
continentes = df_rankings['World regions according to OWID'].unique()
cores = plt.cm.Set3(np.linspace(0, 1, len(continentes)))
for continente, cor in zip(continentes, cores):
    dados = df_rankings[df_rankings['World regions according to OWID'] == continente]
    axes[0, 0].scatter(dados['GDP per capita. PPP (constant 2021 international $)'],
                      dados['Cantril ladder score'],
                      label=continente, alpha=0.6, s=100, color=cor, edgecolors='black', linewidth=0.5)
axes[0, 0].set_xlabel('PIB per Capita (PPP, USD)', fontsize=11)
axes[0, 0].set_ylabel('Felicidade (Escada de Cantril)', fontsize=11)
axes[0, 0].set_title('PIB vs Felicidade por Continente', fontweight='bold', fontsize=12)
axes[0, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
axes[0, 0].grid(True, alpha=0.3)

# IDH vs Felicidade
for continente, cor in zip(continentes, cores):
    dados = df_rankings[df_rankings['World regions according to OWID'] == continente]
    axes[0, 1].scatter(dados['HDI (Human Development Indicator) Value (0~1000)'],
                      dados['Cantril ladder score'],
                      label=continente, alpha=0.6, s=100, color=cor, edgecolors='black', linewidth=0.5)
axes[0, 1].set_xlabel('IDH (0-1000)', fontsize=11)
axes[0, 1].set_ylabel('Felicidade (Escada de Cantril)', fontsize=11)
axes[0, 1].set_title('IDH vs Felicidade por Continente', fontweight='bold', fontsize=12)
axes[0, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
axes[0, 1].grid(True, alpha=0.3)

# IDH vs Homicídios (invertido - menor é melhor)
for continente, cor in zip(continentes, cores):
    dados = df_rankings[df_rankings['World regions according to OWID'] == continente]
    axes[1, 0].scatter(dados['HDI (Human Development Indicator) Value (0~1000)'],
                      dados['Intentional homicides (per 100000 people) as of 2021'],
                      label=continente, alpha=0.6, s=100, color=cor, edgecolors='black', linewidth=0.5)
axes[1, 0].set_xlabel('IDH (0-1000)', fontsize=11)
axes[1, 0].set_ylabel('Taxa de Homicídios (por 100k)', fontsize=11)
axes[1, 0].set_title('IDH vs Violência por Continente', fontweight='bold', fontsize=12)
axes[1, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
axes[1, 0].grid(True, alpha=0.3)

# PIB vs Homicídios
for continente, cor in zip(continentes, cores):
    dados = df_rankings[df_rankings['World regions according to OWID'] == continente]
    axes[1, 1].scatter(dados['GDP per capita. PPP (constant 2021 international $)'],
                      dados['Intentional homicides (per 100000 people) as of 2021'],
                      label=continente, alpha=0.6, s=100, color=cor, edgecolors='black', linewidth=0.5)
axes[1, 1].set_xlabel('PIB per Capita (PPP, USD)', fontsize=11)
axes[1, 1].set_ylabel('Taxa de Homicídios (por 100k)', fontsize=11)
axes[1, 1].set_title('PIB vs Violência por Continente', fontweight='bold', fontsize=12)
axes[1, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Relações entre Indicadores Socioeconômicos por Continente', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('relacoes_indicadores_continentes.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico de relações salvo em 'relacoes_indicadores_continentes.png'")


In [ ]:
# Heatmap de correlação por continente
print("\n" + "="*80)
print("🌍 ANÁLISE DE CORRELAÇÕES POR CONTINENTE")
print("="*80)

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.ravel()

continentes_ordenados = sorted(continentes)
for idx, continente in enumerate(continentes_ordenados[:6]):  # Limitar a 6 para visualização
    dados_cont = df_rankings[df_rankings['World regions according to OWID'] == continente]
    if len(dados_cont) > 2:  # Precisa de pelo menos 3 pontos para correlação
        cols_corr = ['Cantril ladder score', 
                    'GDP per capita. PPP (constant 2021 international $)',
                    'HDI (Human Development Indicator) Value (0~1000)',
                    'Intentional homicides (per 100000 people) as of 2021']
        corr_cont = dados_cont[cols_corr].corr()
        
        mask = np.triu(np.ones_like(corr_cont, dtype=bool))
        sns.heatmap(corr_cont, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
                   center=0, square=True, ax=axes[idx], cbar_kws={"shrink": 0.8},
                   vmin=-1, vmax=1, linewidths=0.5)
        axes[idx].set_title(f'{continente}\n({len(dados_cont)} países)', 
                           fontweight='bold', fontsize=10)
    else:
        axes[idx].text(0.5, 0.5, f'{continente}\nDados insuficientes', 
                      ha='center', va='center', fontsize=12)
        axes[idx].set_xticks([])
        axes[idx].set_yticks([])

plt.suptitle('Matriz de Correlação por Continente', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('correlacoes_por_continente.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Heatmaps de correlação por continente salvos em 'correlacoes_por_continente.png'")


In [ ]:
# Dashboard de Top 10 em diferentes categorias
print("\n" + "="*80)
print("📈 DASHBOARD VISUAL: TOP 10 EM DIFERENTES CATEGORIAS")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Top 10 Mais Ricos
top10_ricos_viz = df_rankings.nlargest(10, 'GDP per capita. PPP (constant 2021 international $)')
axes[0, 0].barh(range(len(top10_ricos_viz)), 
                top10_ricos_viz['GDP per capita. PPP (constant 2021 international $)'],
                color='green', edgecolor='black', alpha=0.8)
axes[0, 0].set_yticks(range(len(top10_ricos_viz)))
axes[0, 0].set_yticklabels(top10_ricos_viz['Country Name'], fontsize=9)
axes[0, 0].set_xlabel('PIB per Capita (USD)', fontsize=11)
axes[0, 0].set_title('💰 Top 10 Países Mais Ricos', fontweight='bold', fontsize=12)
axes[0, 0].grid(True, alpha=0.3, axis='x')
axes[0, 0].invert_yaxis()

# Top 10 Mais Felizes
top10_felizes_viz = df_rankings.nlargest(10, 'Cantril ladder score')
axes[0, 1].barh(range(len(top10_felizes_viz)), 
                top10_felizes_viz['Cantril ladder score'],
                color='gold', edgecolor='black', alpha=0.8)
axes[0, 1].set_yticks(range(len(top10_felizes_viz)))
axes[0, 1].set_yticklabels(top10_felizes_viz['Country Name'], fontsize=9)
axes[0, 1].set_xlabel('Felicidade (Escada de Cantril)', fontsize=11)
axes[0, 1].set_title('😊 Top 10 Países Mais Felizes', fontweight='bold', fontsize=12)
axes[0, 1].grid(True, alpha=0.3, axis='x')
axes[0, 1].invert_yaxis()

# Top 10 Maior IDH
top10_idh_viz = df_rankings.nlargest(10, 'HDI (Human Development Indicator) Value (0~1000)')
axes[1, 0].barh(range(len(top10_idh_viz)), 
                top10_idh_viz['HDI (Human Development Indicator) Value (0~1000)'],
                color='steelblue', edgecolor='black', alpha=0.8)
axes[1, 0].set_yticks(range(len(top10_idh_viz)))
axes[1, 0].set_yticklabels(top10_idh_viz['Country Name'], fontsize=9)
axes[1, 0].set_xlabel('IDH (0-1000)', fontsize=11)
axes[1, 0].set_title('🌍 Top 10 Países com Maior IDH', fontweight='bold', fontsize=12)
axes[1, 0].grid(True, alpha=0.3, axis='x')
axes[1, 0].invert_yaxis()

# Top 10 Mais Seguros (menor taxa de homicídios)
top10_seguros_viz = df_rankings.nsmallest(10, 'Intentional homicides (per 100000 people) as of 2021')
axes[1, 1].barh(range(len(top10_seguros_viz)), 
                100 - top10_seguros_viz['Intentional homicides (per 100000 people) as of 2021'],
                color='lightgreen', edgecolor='black', alpha=0.8)
axes[1, 1].set_yticks(range(len(top10_seguros_viz)))
axes[1, 1].set_yticklabels(top10_seguros_viz['Country Name'], fontsize=9)
axes[1, 1].set_xlabel('Índice de Segurança (invertido)', fontsize=11)
axes[1, 1].set_title('🛡️ Top 10 Países Mais Seguros', fontweight='bold', fontsize=12)
axes[1, 1].grid(True, alpha=0.3, axis='x')
axes[1, 1].invert_yaxis()

plt.suptitle('Dashboard: Top 10 Rankings em Diferentes Categorias', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('dashboard_top10.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Dashboard de Top 10 salvo em 'dashboard_top10.png'")


In [ ]:
# Resumo Executivo das Análises Exploratórias
print("\n" + "="*80)
print("📋 RESUMO EXECUTIVO DAS ANÁLISES EXPLORATÓRIAS")
print("="*80)

resumo = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RESUMO EXECUTIVO: INSIGHTS EXTRAÍDOS DOS DADOS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. RANKING DE CONTINENTES:
   🥇 {ranking_continentes.iloc[0]['Continente']} - Melhor score de desenvolvimento
   🥈 {ranking_continentes.iloc[1]['Continente']} - Segundo melhor
   🥉 {ranking_continentes.iloc[2]['Continente']} - Terceiro melhor

2. INSIGHTS PRINCIPAIS:

   💰 RIQUEZA:
   - País mais rico: {top10_ricos.iloc[0]['Country Name']} (${top10_ricos.iloc[0]['GDP per capita. PPP (constant 2021 international $)']:,.0f})
   - País mais pobre: {top10_pobres.iloc[0]['Country Name']} (${top10_pobres.iloc[0]['GDP per capita. PPP (constant 2021 international $)']:,.0f})
   - Diferença: {top10_ricos.iloc[0]['GDP per capita. PPP (constant 2021 international $)'] / top10_pobres.iloc[0]['GDP per capita. PPP (constant 2021 international $)']:.1f}x

   😊 FELICIDADE:
   - País mais feliz: {top10_felizes.iloc[0]['Country Name']} (Felicidade: {top10_felizes.iloc[0]['Cantril ladder score']:.2f})
   - País menos feliz: {top10_infelizes.iloc[0]['Country Name']} (Felicidade: {top10_infelizes.iloc[0]['Cantril ladder score']:.2f})
   - Diferença: {top10_felizes.iloc[0]['Cantril ladder score'] - top10_infelizes.iloc[0]['Cantril ladder score']:.2f} pontos

   🌍 DESENVOLVIMENTO HUMANO:
   - Maior IDH: {top10_idh.iloc[0]['Country Name']} (IDH: {top10_idh.iloc[0]['HDI (Human Development Indicator) Value (0~1000)']:.0f})
   - Menor IDH: {top10_idh_baixo.iloc[0]['Country Name']} (IDH: {top10_idh_baixo.iloc[0]['HDI (Human Development Indicator) Value (0~1000)']:.0f})

   🛡️ SEGURANÇA:
   - País mais seguro: {top10_seguros.iloc[0]['Country Name']} ({top10_seguros.iloc[0]['Intentional homicides (per 100000 people) as of 2021']:.2f} homicídios/100k)
   - País mais violento: {top10_violentos.iloc[0]['Country Name']} ({top10_violentos.iloc[0]['Intentional homicides (per 100000 people) as of 2021']:.2f} homicídios/100k)
   - Diferença: {top10_violentos.iloc[0]['Intentional homicides (per 100000 people) as of 2021'] / top10_seguros.iloc[0]['Intentional homicides (per 100000 people) as of 2021']:.1f}x mais violento

3. DESCOBERTAS INTERESSANTES:

   💡 Dinheiro vs Felicidade:
   - Países ricos mas menos felizes: {', '.join(ricos_infelizes['Country Name'].head(3).tolist())}
   - Países pobres mas mais felizes: {', '.join(pobres_felizes['Country Name'].head(3).tolist())}
   - Conclusão: Riqueza não garante felicidade, e felicidade não requer riqueza extrema

   🏆 Melhor Equilíbrio:
   - Top 3 países com melhor equilíbrio geral: {', '.join(top_equilibrio['Country Name'].head(3).tolist())}
   - Estes países combinam alta riqueza, felicidade, desenvolvimento humano e segurança

4. PODER DA CIÊNCIA DE DADOS:

   ✅ Esta análise demonstra como dados podem revelar:
   - Padrões geográficos (diferenças entre continentes)
   - Relações complexas (riqueza vs felicidade)
   - Rankings objetivos em múltiplas dimensões
   - Exceções interessantes (países que desafiam expectativas)
   - Oportunidades para políticas públicas baseadas em evidências

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(resumo)
